# ベクトル検索の実装 (Amazon Bedrock 編)<a id="vector-search-with-sagemaker"></a>

> この章は、`vector-search-with-sagemaker.ipynb` を元に作成しています。

## 概要
本ラボでは、Amazon Bedrock の密ベクトル埋め込みモデルを活用したベクトル検索を実装していきます。

### ベクトル
ベクトル検索とは、与えられたクエリアイテムに類似または関連するアイテムを効率的かつ効果的に検索する手法です。ベクトル間の距離や角度の近さといった数値に基づき、類似のアイテムやエンティティを探査します。従来の検索エンジンが苦手とする類似表現や関連語を含むクエリによる問い合わせでも類似性の高い結果を返すことができるため、レコメンドや類似検索、検索検索拡張生成(RAG) に代表される文書検索・ナレッジ検索で幅広く活用されています。

全文検索はクエリと検索対象のデータ間で厳密なマッチングが要求される一方、ベクトル検索は"意味的に近い" 文書を取得する際に有用であるため、うまく使い分けることで幅広い検索要件を達成できます。

一般的にベクトル検索とは、N 次元の数値配列からなる密ベクトルを使った検索のことを指します。密ベクトル検索では、クエリと検索対象のデータは N 次元の数値配列として扱われ、それらの距離や角度の差異が類似度として表されます。距離や角度が近いほど類似度が高いとみなすことができます。

<img src="./img/dense-vector-search.png" width="1024">

### ベクトル埋め込み
ベクトル検索を行う上では、検索対象のテキストやクエリ文字列をベクトルデータに変換し、格納する必要があります。

<img src="./img/dense-vector-embedding.png" width="1024">

データをベクトルに変換する処理を "埋め込み (Embedding)" と呼びます。埋め込み処理は、一般的に機械学習モデルの一種である埋め込みモデル (Embedding model) によって生成します。

本ラボでは、Bedrock の `Amazon Titan Text Embeddings V2` を用いて、テキストからベクトルデータを生成します。モデルの詳細については [Hugging Face 上の解説](https://huggingface.co/amazon/Titan-text-embeddings-v2) を参照してください。

### k-NN search
OpenSearch においては、ベクトル検索を実行するために [k-NN search(k-nearest neighbors search)][knn] と呼ばれる機能を提供しています。k-NN search は、ベクトル空間内で最も近い k 個の近傍点を探す機能です。

k-NN search では、データセットの規模や要件に応じた複数の方式を提供しています。大規模データには Approximate k-NN、フィルタリングが必要な小規模データには Script Score k-NN、複雑なスコアリングが必要な場合は Painless extensions が推奨されてます。

- Approximate k-NN：大規模データセット向けの近似検索方式です。インデックス作成速度と検索精度を多少犠牲にする代わりに、低レイテンシーと少ないメモリ使用量を実現します。
- Script Score k-NN：完全一致の総当たり検索を行う方式です。フィルタリングと組み合わせた検索が可能です。小規模データセット向きです。
- Painless extensions：距離関数をPainlessスクリプトの拡張として提供し、より複雑なスコアリングが可能です。

本ラボでは、実ユースケースでも多く採用されている Approximate kNN を使用して k-NN search を実行していきます。

### Approximate k-NN
[Approximate k-NN](https://opensearch.org/docs/latest/search-plugins/knn/approximate-knn/) (近似最近傍探索、ANN)は、大規模なデータセットで効率的な類似検索を実現するための手法です。Script score による厳密な k-NN search はクエリと全てのデータポイント間の距離を総当たりで計算するため、高次元の大規模データセットでは処理効率が低下します。

ANN は、グラフやバケットなど独自のデータ構造にベクトルデータを格納することで、検索速度を大幅に向上させるアプローチです。精度が若干低下するものの、大規模なベクトルデータに対して効率的な検索が可能になります。

OpenSearch では、ANN を実行するためのエンジンを以下 3 つ用意しています。ただし nmslib は将来廃止予定であるため、実質的には Faiss と Lucene のどちらかから選択する形となります。Faiss は高機能かつ高速であり、大規模データセットに適しています。Lucene は省リソースが特徴で、数百万ベクトルまでの小規模なデータセットで良好な性能を発揮します。

- Faiss (デフォルト)
- Lucene
- nmslib (将来廃止予定)

本ラボでは、実ユースケースでも多く採用されている Faiss エンジンを使用します。

もう一つ ANN を使用するうえで必要になるのがアルゴリズムの選定です。OpenSearch では、以下 2 つのアルゴリズムを提供しています。HNSW は多くのメモリを必要としますが、高速な検索が可能です。IVF はメモリ効率が良好ですが、検索にあたっては事前トレーニングが必要です。各アルゴリズムの詳細については、AWS Bigdata blog の [OpenSearch における 10 億規模のユースケースに適した k-NN アルゴリズムの選定
](https://aws.amazon.com/jp/blogs/news/choose-the-k-nn-algorithm-for-your-billion-scale-use-case-with-opensearch/)に詳しい解説が掲載されています。

- HNSW (Hierarchical Navigable Small World)
- IVF (Inverted File Index)

本ラボでは、トレーニングが不要な HNSW を使用します。

### OpenSearch におけるベクトル検索の流れ
OpenSearch におけるベクトル検索の流れは以下の通りです。

**ベクトルデータの登録**
1. ドキュメントデータをもとに埋め込みモデルを呼び出し、ベクトルを作成
1. ベクトル情報と元のドキュメントデータを検索インデックスに登録

**検索**
1. ユーザーから入力されたクエリをもとに埋め込みモデルを呼び出し、ベクトルを作成
1. ベクトル情報を元にベクトル検索のクエリを組み立て、検索インデックスにベクトル検索のクエリを発行

<img src="./img/dense-vector-embedding-overview.png" width="1024">

### ラボの構成
本ラボでは、ノートブック環境（EC2 へ Remote Develop 接続した VSCode）および Amazon OpenSearch Serverless、Amazon Bedrock を使用します。

<img src="./img/architecture-with-bedrock.png" width="50%" style="display: block; margin: auto;">

### 使用するデータセット
本ラボでは、[JGLUE][jglue] 内の FAQ データセットである [JSQuAD][jsquad] を使用します。

[jglue]: https://github.com/yahoojapan/JGLUE/tree/main
[jsquad]: https://github.com/yahoojapan/JGLUE/tree/main/datasets/jsquad-v1.3
[bge-m3]: https://huggingface.co/BAAI/bge-m3
[knn]: https://opensearch.org/docs/latest/search-plugins/knn/index/

## 事前作業

### パッケージインストール
実行する前に、タブの右上のカーネルの選択を確認してください。

> opensearch と awswrangler のバージョンの依存関係のため、opensearch-py 側を止めています。

In [1]:
!uv add "opensearch-py<3" requests-aws4auth awswrangler[opensearch] tqdm

Resolved 63 packages in 24ms
Audited 56 packages in 0.01ms


### 環境変数
`.env`ファイルに、以下の環境変数を設定します。

- AOSS_SEARCH_HOST=`AOSS の検索コレクションのエンドポイントのホスト名`
- AOSS_VECTOR_HOST=`AOSS のベクトル検索コレクションのエンドポイントのホスト名`
- AOSS_ROLE_ARN=`AOSS から Bedrock へアクセスするために作成したロールのARN`

設定できたら、以下のコマンドを実行します。

In [24]:
%reload_ext dotenv
%dotenv -o

### インポート

In [27]:
import boto3
import json
import time
import logging

from tqdm import tqdm

import awswrangler as wr
import pandas as pd
import numpy as np
from opensearchpy import OpenSearch, RequestsHttpConnection, AWSV4SignerAuth

import os

### 共通変数のセット

In [4]:
default_region = boto3.Session().region_name
logging.getLogger().setLevel(logging.ERROR)

## リソースの準備
### Amazon Bedrock 関連リソースの準備
#### 埋め込みモデルの準備
ここまで、モデルの許可のみ

##### 推論エンドポイントのテスト呼び出し
[このコード](https://docs.aws.amazon.com/ja_jp/bedrock/latest/userguide/bedrock-runtime_example_bedrock-runtime_InvokeModelWithResponseStream_TitanTextEmbeddings_section.html)を参考に実装する

In [5]:
# Create a Bedrock Runtime client in the AWS Region of your choice.
bedrock_runtime_client = boto3.client("bedrock-runtime", region_name=default_region)

# Set the model ID, e.g., Titan Text Embeddings V2.
model_id = "amazon.titan-embed-text-v2:0"

# The text to convert to an embedding.
input_text = "Please recommend books with a theme similar to the movie 'Inception'."

# Create the request for the model.
native_request = {"inputText": input_text}

# Convert the native request to JSON.
request = json.dumps(native_request)

# Invoke the model with the request.
response = bedrock_runtime_client.invoke_model(modelId=model_id, body=request)

# Decode the model's native response body.
model_response = json.loads(response["body"].read())

# Extract and print the generated embedding and the input text token count.
embedding = model_response["embedding"]
input_token_count = model_response["inputTextTokenCount"]

print("\nYour input:")
print(input_text)
print(f"Number of input tokens: {input_token_count}")
print(f"Size of the generated embedding: {len(embedding)}")
print("Embedding:")
print(embedding)


Your input:
Please recommend books with a theme similar to the movie 'Inception'.
Number of input tokens: 15
Size of the generated embedding: 1024
Embedding:
[-0.024926796555519104, 0.042596425861120224, 0.009584195911884308, 0.023822445422410965, -0.0003919462615158409, -0.04417407140135765, 0.011359047144651413, -0.033761609345674515, -0.024453504011034966, -0.03360384702682495, -0.04953806474804878, 0.014908748678863049, -0.032026201486587524, -0.015303160063922405, 0.03944113478064537, -0.017906274646520615, -0.03865231201052666, 0.0329727903008461, 0.04890700802206993, 0.008282638154923916, 0.07762015610933304, 0.06910086423158646, -0.03186843544244766, 0.010964635759592056, -0.0040624369867146015, 0.025557855144143105, 0.07036298513412476, -0.027293264865875244, 0.01877398043870926, -0.005955611355602741, 0.008045991882681847, 0.019878331571817398, 0.04985359311103821, -0.006350022740662098, 5.515596058103256e-05, -0.017985157668590546, 0.0659455806016922, 0.02760879509150982, 0

### サンプルデータの読み込み
サンプルデータをダウンロードし、Pandas の DataFrame 形式に変換します

In [6]:
%%time
dataset_dir = "./dataset/jsquad/"
%mkdir -p $dataset_dir
!curl -L -s -o $dataset_dir/valid.json https://github.com/yahoojapan/JGLUE/raw/main/datasets/jsquad-v1.3/valid-v1.3.json 

CPU times: user 828 μs, sys: 21.6 ms, total: 22.4 ms
Wall time: 1.53 s


In [7]:
%%time
import pandas as pd
import json

def squad_json_to_dataframe(input_file_path, record_path=["data", "paragraphs", "qas", "answers"]):
    file = json.loads(open(input_file_path).read())
    m = pd.json_normalize(file, record_path[:-1])
    r = pd.json_normalize(file, record_path[:-2])

    idx = np.repeat(r["context"].values, r.qas.str.len())
    m["context"] = idx
    m["answers"] = m["answers"]
    m["answers"] = m["answers"].apply(lambda x: np.unique(pd.json_normalize(x)["text"].to_list()))
    return m[["id", "question", "context", "answers"]]

valid_filename = f"{dataset_dir}/valid.json"
valid_df = squad_json_to_dataframe(valid_filename)

CPU times: user 630 ms, sys: 5.62 ms, total: 636 ms
Wall time: 649 ms


### サンプルデータの確認
サンプルデータは質問文フィールドの question、回答の answers、説明文の context フィールド、問題 ID である id フィールドから構成されています。

サンプルデータの一部を見ていきましょう。

In [8]:
valid_df

,id,question,context,answers
0,a10336p0q0,日本で梅雨がないのは北海道とどこか。,梅雨 [SEP] 梅雨（つゆ、ばいう）は、北海道と小笠原諸島を除く日本、朝鮮半島南部、中国の...,"[小笠原諸島, 小笠原諸島を除く日本]"
1,a10336p0q1,梅雨とは何季の一種か?,梅雨 [SEP] 梅雨（つゆ、ばいう）は、北海道と小笠原諸島を除く日本、朝鮮半島南部、中国の...,[雨季]
2,a10336p0q2,梅雨は、世界的にどのあたりで見られる気象ですか？,梅雨 [SEP] 梅雨（つゆ、ばいう）は、北海道と小笠原諸島を除く日本、朝鮮半島南部、中国の...,"[東アジア, 東アジアの広範囲]"
3,a10336p0q3,梅雨がみられるのはどの期間？,梅雨 [SEP] 梅雨（つゆ、ばいう）は、北海道と小笠原諸島を除く日本、朝鮮半島南部、中国の...,"[5月から7月, 5月から7月にかけて]"
4,a10336p1q0,入梅は何の目安の時期か？,梅雨 [SEP] 梅雨の時期が始まることを梅雨入りや入梅（にゅうばい）といい、社会通念上・気...,"[春の終わりであるとともに夏の始まり（初夏）, 田植えの時期, 田植えの時期の目安]"
...,...,...,...,...
4437,a95156p5q3,国際銀行間通信協会ならびに国際決済機関の何と何も企業体である,多国籍企業 [SEP] 国際銀行間通信協会ならびに国際決済機関のクリアストリームとユーロクリ...,[クリアストリームとユーロクリア]
4438,a95156p6q0,ゼネコンはどの国特有の形態か,多国籍企業 [SEP] ゼネコンは日本特有の形態。セメントメジャーにラファージュホルシムやイ...,[日本]
4439,a95156p6q1,多国籍企業においてゼネコンはどこの国特有の形態であるか？,多国籍企業 [SEP] ゼネコンは日本特有の形態。セメントメジャーにラファージュホルシムやイ...,[日本]
4440,a95156p6q2,多国籍企業を一つ挙げよ,多国籍企業 [SEP] ゼネコンは日本特有の形態。セメントメジャーにラファージュホルシムやイ...,"[イタルチェメンティ, ラファージュホルシム]"


### OpenSearch Serverless への接続確認
OpenSearch Server のセキュリティ設定により、API リクエストが許可されているかを確認します。

In [9]:
aoss_host = os.getenv("AOSS_VECTOR_HOST")

credentials = boto3.Session().get_credentials()
service_code = "aoss"
auth = AWSV4SignerAuth(credentials=credentials, region=default_region, service=service_code)
opensearch_client = OpenSearch(
    hosts=[{"host": aoss_host, "port": 443}],
    http_compress=True, 
    http_auth=auth,
    use_ssl=True,
    verify_certs=True,
    connection_class = RequestsHttpConnection
)
opensearch_client.cat.indices()

' OPEN jsquad-neural-search 8ZM-8JgBcgEDf7NL50Kx   4442 0 137.1mb 137.1mb\n OPEN jsquad-knn           O_IP-pgB_vf32jNh0AOA      0 0      0b      0b\n'

#### インデックスの作成
id、question、context、answers フィールドを格納するための文字列型フィールドに加えて、question、context フィールドから生成したベクトルデータを格納するための context_embedding、question_embedding フィールドを持つインデックスを作成します。

question、context、answers フィールドについては、テキスト検索でもある程度の検索精度を出せるように、id フィールドを除いて kuromoji のカスタムアナライザーをセットしています。

OpenSearch では、ベクトルデータを格納するためのフィールドタイプとして knn_vector タイプを提供しています。

In [10]:
payload = {
  "mappings": {
    "properties": {
      "id": {"type": "keyword"},
      "question": {"type": "text", "analyzer": "custom_kuromoji_analyzer"},
      "context":  {"type": "text", "analyzer": "custom_kuromoji_analyzer"},
      "answers":  {"type": "text", "analyzer": "custom_kuromoji_analyzer"},
      "question_embedding": {
        "type": "knn_vector",
        "dimension": 1024,
        "space_type": "l2",
        "method": {
          "name": "hnsw",
          "engine": "faiss",
        }
      },
      "context_embedding": {
        "type": "knn_vector",
        "dimension": 1024,
        "space_type": "l2",
        "method": {
          "name": "hnsw",
          "engine": "faiss",
        },
      }
    }
  },
  "settings": {
    "index.knn": True,
    "analysis": {
      "analyzer": {
        "custom_kuromoji_analyzer": {
          "char_filter": ["icu_normalizer"],
          "filter": [
              "kuromoji_baseform",
              "custom_kuromoji_part_of_speech"
          ],
          "tokenizer": "kuromoji_tokenizer",
          "type": "custom"
        }
      },
      "filter": {
        "custom_kuromoji_part_of_speech": {
          "type": "kuromoji_part_of_speech",
          "stoptags": ["感動詞,フィラー","接頭辞","代名詞","副詞","助詞","助動詞","動詞,一般,*,*,*,終止形-一般","名詞,普通名詞,副詞可能"]
        }
      }
    }
  }
}
# インデックス名を指定
index_name = "jsquad-knn"

try:
    # 既に同名のインデックスが存在する場合、いったん削除を行う
    print("# delete index")
    response = opensearch_client.indices.delete(index=index_name)
    print(json.dumps(response, indent=2))
except Exception as e:
    print(e)

# インデックスを作成
response = opensearch_client.indices.create(index_name, body=payload)
response

# delete index
{
  "acknowledged": true
}


{'acknowledged': True, 'shards_acknowledged': True, 'index': 'jsquad-knn'}

### サンプルデータの格納
サンプルデータにベクトルデータを追加し、OpenSearch に格納します。

#### ベクトルデータを生成(埋め込み)
DataFrame 形式に加工したサンプルデータの question フィールドと context フィールドを対象に、Bedrock 上で稼働している埋め込みモデルの推論エンドポイントを呼び出してベクトルデータを生成、結合する処理を実行します。

<div class="alert alert-block alert-warning"> 
Bedrock での Batch 実行は、S3 経由であり、メモリ中のデータを invoke_model で処理する場合は、１件ずつ処理されるので、15分前後かかります。
</div>

In [11]:
%%time

def get_df_with_embeddings(input_df, field_mappings, model_id, bedrock_region, batch_size):
    output_df = pd.DataFrame([]) #create empty dataframe
    df_list = np.array_split(input_df, input_df.shape[0]/batch_size)
    for df in tqdm(df_list):
        index = df.index #backup index number
        df_with_embeddings = df
        for field_mapping in field_mappings:
            input_field_name = field_mapping["InputFieldName"]
            embedding_field_name = field_mapping["EmbeddingFieldName"]
            payload = {
                "inputText": df_with_embeddings[input_field_name].values.tolist()[0]
            }
            body = json.dumps(payload)
            bedrock_runtime_client = boto3.client("bedrock-runtime", region_name=bedrock_region)
            response = bedrock_runtime_client.invoke_model(modelId=model_id, body=body)
            model_response = json.loads(response["body"].read())
            embeddings = [model_response["embedding"]]

            df_with_embeddings = pd.concat([df_with_embeddings.reset_index(drop=True), pd.Series(embeddings,name=embedding_field_name).reset_index(drop=True)],axis=1) #join embedding results to source dataframe
            df_with_embeddings = df_with_embeddings.set_index(index) #restore index number

        output_df = pd.concat([output_df, df_with_embeddings])
    return output_df

valid_df_with_embeddings = get_df_with_embeddings(
    input_df=valid_df,
    field_mappings=[
        {"InputFieldName": "question", "EmbeddingFieldName": "question_embedding"},
        {"InputFieldName": "context", "EmbeddingFieldName": "context_embedding"},
    ],
    model_id=model_id,
    bedrock_region=default_region,
    batch_size=1
)

/home/ec2-user/work/aoss/aoss-bedrock-workshop/.venv/lib/python3.13/site-packages/numpy/_core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
100%|██████████| 4442/4442 [14:03<00:00,  5.27it/s]


CPU times: user 3min 51s, sys: 3.92 s, total: 3min 55s
Wall time: 14min 3s


実行後の DataFrame は以下の通りです。数値配列の question_embedding フィールドおよび context_embedding フィールドが追加されていることが確認できます。

In [12]:
valid_df_with_embeddings

,id,question,context,answers,question_embedding,context_embedding
0,a10336p0q0,日本で梅雨がないのは北海道とどこか。,梅雨 [SEP] 梅雨（つゆ、ばいう）は、北海道と小笠原諸島を除く日本、朝鮮半島南部、中国の...,"[小笠原諸島, 小笠原諸島を除く日本]","[-0.030808305367827415, 0.01807420700788498, -...","[-0.007869734428822994, 0.06828266382217407, -..."
1,a10336p0q1,梅雨とは何季の一種か?,梅雨 [SEP] 梅雨（つゆ、ばいう）は、北海道と小笠原諸島を除く日本、朝鮮半島南部、中国の...,[雨季],"[-0.03151561692357063, 0.03348534554243088, 0....","[-0.007869734428822994, 0.06828266382217407, -..."
2,a10336p0q2,梅雨は、世界的にどのあたりで見られる気象ですか？,梅雨 [SEP] 梅雨（つゆ、ばいう）は、北海道と小笠原諸島を除く日本、朝鮮半島南部、中国の...,"[東アジア, 東アジアの広範囲]","[-0.03327362984418869, 0.05146321654319763, 0....","[-0.007869734428822994, 0.06828266382217407, -..."
3,a10336p0q3,梅雨がみられるのはどの期間？,梅雨 [SEP] 梅雨（つゆ、ばいう）は、北海道と小笠原諸島を除く日本、朝鮮半島南部、中国の...,"[5月から7月, 5月から7月にかけて]","[-0.051503986120224, 0.004789622034877539, 0.0...","[-0.007869734428822994, 0.06828266382217407, -..."
4,a10336p1q0,入梅は何の目安の時期か？,梅雨 [SEP] 梅雨の時期が始まることを梅雨入りや入梅（にゅうばい）といい、社会通念上・気...,"[春の終わりであるとともに夏の始まり（初夏）, 田植えの時期, 田植えの時期の目安]","[-0.016133945435285568, 0.022329380735754967, ...","[-0.03153686597943306, 0.018423322588205338, -..."
...,...,...,...,...,...,...
4437,a95156p5q3,国際銀行間通信協会ならびに国際決済機関の何と何も企業体である,多国籍企業 [SEP] 国際銀行間通信協会ならびに国際決済機関のクリアストリームとユーロクリ...,[クリアストリームとユーロクリア],"[-0.06765758246183395, 0.059636324644088745, 0...","[-0.06717027723789215, 0.029904576018452644, 0..."
4438,a95156p6q0,ゼネコンはどの国特有の形態か,多国籍企業 [SEP] ゼネコンは日本特有の形態。セメントメジャーにラファージュホルシムやイ...,[日本],"[-0.03447365388274193, 0.03243077173829079, -0...","[-0.03695594146847725, 0.05384530872106552, -0..."
4439,a95156p6q1,多国籍企業においてゼネコンはどこの国特有の形態であるか？,多国籍企業 [SEP] ゼネコンは日本特有の形態。セメントメジャーにラファージュホルシムやイ...,[日本],"[-0.033731359988451004, 0.039792466908693314, ...","[-0.03695594146847725, 0.05384530872106552, -0..."
4440,a95156p6q2,多国籍企業を一つ挙げよ,多国籍企業 [SEP] ゼネコンは日本特有の形態。セメントメジャーにラファージュホルシムやイ...,"[イタルチェメンティ, ラファージュホルシム]","[-0.038366202265024185, 0.057634562253952026, ...","[-0.03695594146847725, 0.05384530872106552, -0..."


#### ドキュメントのロード
ドキュメントのロードを行います。ドキュメントのロードは "OpenSearch の基本概念・基本操作の理解" でも解説した通り bulk API を使用することで効率よく進められますが、データ処理フレームワークを利用することでより簡単にデータを取り込むことも可能です。本ワークショップでは、[AWS SDK for Pandas][aws-sdk-pandas] を使用したデータ取り込みを行います。

[aws-sdk-pandas]: https://github.com/aws/aws-sdk-pandas

<div class="alert alert-block alert-warning"> 
AOSS のベクトル検索コレクションの場合、ID 指定できません。<br>
指定すると、`Document ID is not supported in create/index operation request` となります。
</div>

In [13]:
%%time
index_name = "jsquad-knn"
response = wr.opensearch.index_df(
    client=opensearch_client,
    df=valid_df_with_embeddings,
    use_threads=True,
    index=index_name,
    bulk_size=200, # 200 件ずつ書き込み
    refresh=False,
)

CPU times: user 53.2 s, sys: 185 ms, total: 53.4 s
Wall time: 1min 3s


response["success"] の値が DataFrame の件数と一致しているかを確認します。<br>
True が表示される場合は全件登録に成功していると判断できます。

In [14]:
response["success"] == valid_df["id"].count()

np.True_

## 検索結果の比較
テキスト検索とベクトル検索を実行し、結果を比較していきます。

### テキスト検索
テキスト検索のヒット率は検索キーワードとインデックスに格納されたコンテンツの内容、およびアナライザーによる正規化設定により左右されます。<br>
テキスト検索では極力不要なキーワードは排除して検索が実行されることが好まれます。以下のような単語の組み合わせによる検索で性能を発揮します。

In [15]:
index_name = "jsquad-knn"
query = "日本 梅雨 ない どこ"

payload = {
  "query": {
    "match": {
      "question": {
        "query": query,
        "operator": "and"
      }
    }
  },
  "_source": False,
  "fields": ["question", "answers", "context"],
  "size": 10
}
response = opensearch_client.search(
    index=index_name,
    body=payload
)
pd.json_normalize(response["hits"]["hits"])

,_index,_id,_score,fields.question,fields.answers,fields.context
0,jsquad-knn,1%3A0%3A0NuQ_ZgBQNa1Y6duMPl-,15.38611,[日本で梅雨がないのは北海道とどこか。],"[小笠原諸島, 小笠原諸島を除く日本]",[梅雨 [SEP] 梅雨（つゆ、ばいう）は、北海道と小笠原諸島を除く日本、朝鮮半島南部、中国...


一方、以下のような会話に近いクエリは、ノイズが増加するためうまく処理できない場合があります。

In [16]:
index_name = "jsquad-knn"
query = "日本で梅雨がない場所は？"

payload = {
  "query": {
    "match": {
      "question": {
        "query": query,
        "operator": "and"
      }
    }
  },
  "_source": False,
  "fields": ["question", "answers", "context"],
  "size": 10
}
response = opensearch_client.search(
    index=index_name,
    body=payload
)
response

{'took': 111,
 'timed_out': False,
 '_shards': {'total': 0, 'successful': 0, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 0, 'relation': 'eq'},
  'max_score': None,
  'hits': []}}

従来は、minimum_should_match といった[パラメーター][parameters]によるチューニングを行ってきました。<br>
以下は検索クエリに含まれるトークンのうち 75% がマッチするドキュメントを返却するクエリです。

[parameters]: https://opensearch.org/docs/latest/query-dsl/full-text/match/#parameters

In [17]:
index_name = "jsquad-knn"
query = "日本で梅雨がない場所は？"

payload = {
  "query": {
    "match": {
      "question": {
        "query": query,
        "operator": "or",
        "minimum_should_match": "75%"
      }
    }
  },
  "_source": False,
  "fields": ["question", "answers", "context"],
  "size": 10
}
response = opensearch_client.search(
    index=index_name,
    body=payload
)
pd.json_normalize(response["hits"]["hits"])

,_index,_id,_score,fields.question,fields.answers,fields.context
0,jsquad-knn,1%3A0%3A0NuQ_ZgBQNa1Y6duMPl-,15.388310,[日本で梅雨がないのは北海道とどこか。],"[小笠原諸島, 小笠原諸島を除く日本]",[梅雨 [SEP] 梅雨（つゆ、ばいう）は、北海道と小笠原諸島を除く日本、朝鮮半島南部、中国...
1,jsquad-knn,1%3A0%3AHduQ_ZgBQNa1Y6duMPp-,6.910740,[気象学では一般的に、梅雨がある中国沿海部・朝鮮半島・日本列島の大部分を何に含めるか],"[モンスーン・アジア, モンスーン・アジアに含める。]",[梅雨 [SEP] アジア・モンスーンの影響範囲はさらに東にまで及んでおり、南シナ海を覆う熱...
2,jsquad-knn,1%3A0%3AINuQ_ZgBQNa1Y6duMPp-,6.629898,[日本では各地の地方気象台と何が、数個の都府県をまとめた地域ごとに毎年梅雨入り・梅雨明けの発...,[気象庁],[梅雨 [SEP] 日本では各地の地方気象台・気象庁が、数個の都府県をまとめた地域ごとに毎年...
3,jsquad-knn,1%3A0%3Ac9uQ_ZgBQNa1Y6duMPp-,6.421738,[中部-東部で海水温が低下・西部で上昇するラニーニャ現象が発生したときは、沖縄で梅雨入りが遅...,"[日照時間はやや少なめとなる傾向にある, 日照時間はやや少なめとなる傾向にある。]",[梅雨 [SEP] 統計的にみて、赤道付近の太平洋中部-東部にかけて海水温が上昇・西部で低下...


### ベクトル検索
テキスト検索では対応が難しい会話に近い問い合わせ分をベクトル検索で処理していきます。

OpenSearch では [knn クエリ](https://opensearch.org/docs/latest/search-plugins/knn/approximate-knn/#get-started-with-approximate-k-nn)を使用してベクトル検索を実行します。vector フィールドにはベクトルデータを、k には取得したい近似ベクトルの件数を指定しています。

OpenSearch は knn クエリも分散実行されるため、インデックスの構成によっては k の値と戻りの総件数の値が異なる場合があります。k の値と size の値はそろえることを推奨しています。詳細は [The number of returned results](https://opensearch.org/docs/latest/search-plugins/knn/approximate-knn/#the-number-of-returned-results) を参照してください。

以下のコードでは、クエリテキストを Amazon Bedrock の推論エンドポイントに渡してベクトルデータを生成し、knn クエリの vector パラメーターに渡しています。

In [18]:
index_name = "jsquad-knn"
model_id = "amazon.titan-embed-text-v2:0"
query = "日本で梅雨がない場所は？"

def text_to_embedding(text, region_name, model_id):
    payload = {
        "inputText": text
    }
    body = json.dumps(payload)
    bedrock_runtime_client = boto3.client("bedrock-runtime", region_name)
    response = bedrock_runtime_client.invoke_model(modelId = model_id, body=body)
    model_response = json.loads(response["body"].read())
    return model_response["embedding"]

vector = text_to_embedding(text=query, region_name=default_region, model_id=model_id)
k = 10

payload = {
  "query": {
    "knn": {
      "question_embedding": {
        "vector": vector,
        "k": k
      }
    }
  },
  "_source": False,
  "fields": ["question", "answers", "context"],
  "size": k
}
response = opensearch_client.search(
    index=index_name,
    body=payload
)
pd.json_normalize(response["hits"]["hits"])

,_index,_id,_score,fields.question,fields.answers,fields.context
0,jsquad-knn,1%3A0%3AFNuQ_ZgBQNa1Y6duMPp-,0.832107,[梅雨が日本の中でない地域はどこか。],"[北海道, 東北地方]",[梅雨 [SEP] 年によっては梅雨明けの時期が特定できなかったり、あるいは発表がされないこ...
1,jsquad-knn,1%3A0%3A0NuQ_ZgBQNa1Y6duMPl-,0.828650,[日本で梅雨がないのは北海道とどこか。],"[小笠原諸島, 小笠原諸島を除く日本]",[梅雨 [SEP] 梅雨（つゆ、ばいう）は、北海道と小笠原諸島を除く日本、朝鮮半島南部、中国...
2,jsquad-knn,1%3A0%3ANNuQ_ZgBQNa1Y6duMPp-,0.659169,[気候学的には梅雨はないとされている場所は？],[北海道],[梅雨 [SEP] 実際の気象としては北海道にも道南を中心に梅雨前線がかかることはあるが、平...
3,jsquad-knn,1%3A0%3ANduQ_ZgBQNa1Y6duMPp-,0.658166,[梅雨がないとされている都道府県はどこ？],[北海道],[梅雨 [SEP] 実際の気象としては北海道にも道南を中心に梅雨前線がかかることはあるが、平...
4,jsquad-knn,1%3A0%3A_NuQ_ZgBQNa1Y6duMPl-,0.567400,[日本の地域で本格的な長雨に突入しない場所はどこか。],[北海道],[梅雨 [SEP] 次に梅雨前線は中国の江淮（長江流域・淮河流域）に北上する。6月下旬には華...
5,jsquad-knn,1%3A0%3AX9uQ_ZgBQNa1Y6duMPp-,0.550389,[ほとんど雨が降らない梅雨を何と呼ぶか],"[空梅雨, 空梅雨（からつゆ）]",[梅雨 [SEP] 梅雨の期間中ほとんど雨が降らない場合がある。このような梅雨のことを空梅雨...
6,jsquad-knn,1%3A0%3AYNuQ_ZgBQNa1Y6duMPp-,0.541300,[ほとんど雨が降らない梅雨を何という？],"[空梅雨, 空梅雨（からつゆ）]",[梅雨 [SEP] 梅雨の期間中ほとんど雨が降らない場合がある。このような梅雨のことを空梅雨...
7,jsquad-knn,1%3A0%3AEtuQ_ZgBQNa1Y6duMPp-,0.537503,[年によっては梅雨入りの時期が特定できなかったり、あるいは発表がされないこともある。東・西日...,"[数年に一回, 数年に一回の割合, 数年に一回の割合で起こる]",[梅雨 [SEP] 年によっては梅雨入りの時期が特定できなかったり、あるいは発表がされないこ...
8,jsquad-knn,1%3A0%3AL9uQ_ZgBQNa1Y6duMPp-,0.529294,[日本付近の梅雨期の雨量が最も多い地区は],[九州南部],[梅雨 [SEP] 梅雨の期間はどの地方でも40日から50日前後と大差はないが、期間中の降水...
9,jsquad-knn,1%3A0%3AHduQ_ZgBQNa1Y6duMPp-,0.527282,[気象学では一般的に、梅雨がある中国沿海部・朝鮮半島・日本列島の大部分を何に含めるか],"[モンスーン・アジア, モンスーン・アジアに含める。]",[梅雨 [SEP] アジア・モンスーンの影響範囲はさらに東にまで及んでおり、南シナ海を覆う熱...


## ベクトル検索のスコアリング
ベクトル検索は _score が 1 未満となります。ベクトル検索においては、クエリと対象ドキュメントの距離が近いほど距離の値は小さくなります。<br>
したがって、距離を 0 から 1 の間で正規化したうえで、1 から距離を引いた値をスコア(関連度)としています。

knn search では、上位 k 個のベクトルという条件以外に、こうした距離やスコアを使った絞り込みが可能です。フィルタリングには以下のオプションを使用可能です。これらのオプションは k と併用不可能です。

- min_score
- max_distance

In [ ]:
index_name = "jsquad-knn"
query = "日本で梅雨がない場所は？"

def text_to_embedding(text, region_name, model_id):
    payload = {
        "inputText": text
    }
    body = json.dumps(payload)
    bedrock_runtime_client = boto3.client("bedrock-runtime", region_name)
    response = bedrock_runtime_client.invoke_model(modelId = model_id, body=body)
    model_response = json.loads(response["body"].read())
    return model_response["embedding"]

vector = text_to_embedding(text=query, region_name=default_region, model_id=model_id)

min_score に 0.7 をセットした結果は以下の通りです。<br>
0.7 以上のスコアのベクトルのみが返却されました。

In [19]:
k = 10

payload = {
  "query": {
    "knn": {
      "question_embedding": {
        "vector": vector,
        "min_score": 0.7
      }
    }
  },
  "_source": False,
  "fields": ["question", "answers", "context"],
  "size": k,
}
response = opensearch_client.search(
    index=index_name,
    body=payload
)
pd.json_normalize(response["hits"]["hits"])

,_index,_id,_score,fields.question,fields.answers,fields.context
0,jsquad-knn,1%3A0%3AFNuQ_ZgBQNa1Y6duMPp-,0.832107,[梅雨が日本の中でない地域はどこか。],"[北海道, 東北地方]",[梅雨 [SEP] 年によっては梅雨明けの時期が特定できなかったり、あるいは発表がされないこ...
1,jsquad-knn,1%3A0%3A0NuQ_ZgBQNa1Y6duMPl-,0.828650,[日本で梅雨がないのは北海道とどこか。],"[小笠原諸島, 小笠原諸島を除く日本]",[梅雨 [SEP] 梅雨（つゆ、ばいう）は、北海道と小笠原諸島を除く日本、朝鮮半島南部、中国...


max_distance に 0.3 をセットしても同様の結果が得られます。<br>
`1 - 0.3 = 0.7` に相当するスコアのベクトルが変えるためです。

In [20]:
k = 10

payload = {
  "query": {
    "knn": {
      "question_embedding": {
        "vector": vector,
        "max_distance": 0.3
      }
    }
  },
  "_source": False,
  "fields": ["question", "answers", "context"],
  "size": k, 
}
response = opensearch_client.search(
    index=index_name,
    body=payload
)
pd.json_normalize(response["hits"]["hits"])

,_index,_id,_score,fields.question,fields.answers,fields.context
0,jsquad-knn,1%3A0%3AFNuQ_ZgBQNa1Y6duMPp-,0.832107,[梅雨が日本の中でない地域はどこか。],"[北海道, 東北地方]",[梅雨 [SEP] 年によっては梅雨明けの時期が特定できなかったり、あるいは発表がされないこ...
1,jsquad-knn,1%3A0%3A0NuQ_ZgBQNa1Y6duMPl-,0.828650,[日本で梅雨がないのは北海道とどこか。],"[小笠原諸島, 小笠原諸島を除く日本]",[梅雨 [SEP] 梅雨（つゆ、ばいう）は、北海道と小笠原諸島を除く日本、朝鮮半島南部、中国...


## まとめ
ラボを通して、全文検索では対応が難しいクエリをベクトル検索で処理できることが確認できました。<br>
時間がある方は、続いて以下のラボも実施してみましょう。

- [ニューラル検索の実装 (Amazon Bedrock 編)](#neural-search-with-sagemaker)

## 後片付け

### データセット削除
ダウンロードしたデータセットを削除します。<br>
./dataset ディレクトリ配下に何もない場合は、./dataset ディレクトリも合わせて削除します。

In [21]:
%rm -rf {dataset_dir}

In [22]:
%rmdir ./dataset

<hr>

# ニューラル検索の実装 (Amazon Bedrock 編)<a id="neural-search-with-sagemaker"></a>
> この章は、`neural-search-with-sagemaker.ipynb` を元に作成しています。

## 概要
本ラボでは、テキストクエリを内部的にベクトルに変換してベクトル検索を行うニューラル検索を実装していきます。

ベクトルの生成は、Amazon SageMaker 上にデプロイした密ベクトル埋め込みモデルを利用します。

### 前提事項
本ラボは、[ベクトル検索の実装 (Amazon Bedrock 編)](#vector-search-with-sagemaker) を完了していることを前提として作られています。

ベクトル検索の基本的な要素の解説や、ML モデルのデプロイなど、本ラボを実施するにあたって必要な作業も含まれているため、事前に上記のラボを完了させてください。

### ニューラル検索について
ニューラル検索は、OpenSearch に入力されたテキストや画像のクエリデータを、OpenSearch 側でベクトルに変換し、登録と検索を実行する機能です。

一般的なベクトル検索は、クライアント側で用意したベクトルを使用したデータ登録や検索を行う必要があります。従来のテキスト検索に加えてベクトル検索を実装しようとする場合、バックエンド側に埋め込みモデルを呼び出す処理を実装する必要があります。

<img src="./img/dense-vector-embedding-with-backend.png" width="1024">

ニューラル検索では、OpenSearch がバックエンドの責務も担います。OpenSearch はユーザークエリをもとに埋め込みモデルを呼び出し、ベクトルの生成を行います。生成したベクトルは、そのまま格納、ないしはベクトル検索に使用します。

ニューラル検索を活用することで、クライアント側の改修を最小限に抑えつつベクトル検索を導入可能となります。

<img src="./img/dense-vector-embedding-with-connector.png" width="1024">

ニューラル検索は以下のコンポーネントで構成されています

- モデル([リモートモデル][remote-models]): 外部サービス上にホストされた ML モデル。接続を行うためには、コネクターが必要となる。
- [コネクター][connector]: Amazon SageMaker の推論エンドポイントなど、外部エンドポイントへの接続情報を管理するコンポーネント
- Embedding processor: パイプラインから与えられたデータを ML モデルに渡すためのプロセッサ。テキスト埋め込み用の [Text embedding processor][text-embedding]、テキスト+画像のマルチモーダル埋め込み用の [Text/image embedding processor][text-image-embedding] など、元のデータフォーマットによって異なるプロセッサが存在する。
- [Ingest pipelines][ingest-pipelines]: ドキュメント登録時に加工処理を行うパイプライン。1 つ以上のプロセッサで構成されている。Embedding processor を呼び出すことで、ドキュメントのテキストもしくは画像データが格納されたバイナリフィールドからベクトル埋め込みを生成し、元のテキストとベクトルの両方をk-NNインデックスに保存することが可能。
- [Search pipelines][search-pipelines]: 検索クエリもしくは検索結果の加工処理を行うパイプライン。1 つ以上のプロセッサで構成されている。Embedding processor を呼び出すことで、クエリからベクトル埋め込みを生成し、ベクトルフィールドに対する検索を実行することが可能となる。

### コネクター
コネクターは、外部サービスとの連携の大部分を担っています。サービスのエンドポイントや、OpenSearch から渡されたデータを外部サービス向けのリクエストペイロードに書き換えるための定義、外部サービスから受け取った応答を OpenSearch 向けのフォーマットに書き換えるための定義といった情報を保持しています。

### ラボの構成
本ラボでは、ノートブック環境（EC2 へ Remote Develop 接続した VSCode）および Amazon OpenSearch Serverless、Amazon Bedrock を使用します

<img src="./img/architecture-with-bedrock.png" width="512">

### 使用するデータセット
本ラボでは、[ベクトル検索の実装 (Amazon Bedrock 編)](#vector-search-with-sagemaker) と同様に、[JGLUE][jglue] 内の FAQ データセットである [JSQuAD][jsquad] を使用します。

[remote-models]: https://opensearch.org/docs/latest/ml-commons-plugin/remote-models/index/
[connector]: https://opensearch.org/docs/latest/ml-commons-plugin/remote-models/connectors/
[text-embedding]: https://opensearch.org/docs/latest/ingest-pipelines/processors/text-embedding/
[text-image-embedding]: https://opensearch.org/docs/latest/ingest-pipelines/processors/text-image-embedding/
[ingest-pipelines]: https://opensearch.org/docs/latest/ingest-pipelines/
[search-pipelines]: https://opensearch.org/docs/latest/search-plugins/search-pipelines/index/
[jglue]: https://github.com/yahoojapan/JGLUE/tree/main
[jsquad]: https://github.com/yahoojapan/JGLUE/tree/main/datasets/jsquad-v1.3
[bge-m3]: https://huggingface.co/BAAI/bge-m3



## 事前作業
ベクトル検索の実装で実施済みです。

## リソース準備

ベクトル検索の実装で実施済みです。

### OpenSearch 関連リソースの作成

#### インデックスの作成
id、question、context、answers フィールドを格納するための文字列型フィールドに加えて、question、context フィールドから生成したベクトルデータを格納するための context_dense_embedding、question_sparse_embedding フィールドを持つインデックスを作成します。

文字列型フィールドについては、テキスト検索でもある程度の検索精度を出せるように、id フィールドを除いて kuromoji のカスタムアナライザーをセットしています。


In [23]:
payload = {
  "mappings": {
    "properties": {
      "id": {"type": "keyword"},
      "question": {"type": "text", "analyzer": "custom_kuromoji_analyzer"},
      "context":  {"type": "text", "analyzer": "custom_kuromoji_analyzer"},
      "answers":  {"type": "text", "analyzer": "custom_kuromoji_analyzer"},
      "question_embedding": {
        "type": "knn_vector",
        "dimension": 1024,
        "space_type": "l2",
        "method": {
          "name": "hnsw",
          "engine": "faiss",
        }
      },
      "context_embedding": {
        "type": "knn_vector",
        "dimension": 1024,
        "space_type": "l2",
        "method": {
          "name": "hnsw",
          "engine": "faiss",
        },
      }
    }
  },
  "settings": {
    "index.knn": True,
    "analysis": {
      "analyzer": {
        "custom_kuromoji_analyzer": {
          "char_filter": ["icu_normalizer"],
          "filter": [
              "kuromoji_baseform",
              "custom_kuromoji_part_of_speech"
          ],
          "tokenizer": "kuromoji_tokenizer",
          "type": "custom"
        }
      },
      "filter": {
        "custom_kuromoji_part_of_speech": {
          "type": "kuromoji_part_of_speech",
          "stoptags": ["感動詞,フィラー","接頭辞","代名詞","副詞","助詞","助動詞","動詞,一般,*,*,*,終止形-一般","名詞,普通名詞,副詞可能"]
        }
      }
    }
  }
}
# インデックス名を指定
index_name = "jsquad-neural-search"

try:
    # 既に同名のインデックスが存在する場合、いったん削除を行う
    print("# delete index")
    response = opensearch_client.indices.delete(index=index_name)
    print(json.dumps(response, indent=2))
except Exception as e:
    print(e)

# インデックスを作成
response = opensearch_client.indices.create(index_name, body=payload)
response

# delete index
{
  "acknowledged": true
}


{'acknowledged': True,
 'shards_acknowledged': True,
 'index': 'jsquad-neural-search'}

#### OpenSearch へのモデル登録
Bedrock 上のモデルを呼び出すためのコンポーネントを作成します。

モデルは、コネクタと呼ばれる外部接続を定義したコンポーネントで構成されています。
今回の構成では、モデルは Text Embedding Processor と呼ばれる、入力テキストをベクトルに変換するためのプロセッサーから呼び出されます。

##### コネクタ用 IAM Role ARN の確認
OpenSearch コネクタから AWS サービスに接続する際、任意の IAM ロールの権限を引き受ける必要があります。<br>
引受対象の IAM ロールを CloudFormation スタックの出力から取得します。

In [25]:
opensearch_connector_role_arn = os.getenv("AOSS_ROLE_ARN")
opensearch_connector_role_arn

'arn:aws:iam::720701806597:role/TestAossToBedrockRole'

##### コネクタの作成
Amazon Bedrock 上のモデルを呼び出す定義を記載したコネクタを作成します。<br>
コネクタは、OpenSearch におけるモデルの一要素です。

コネクタの処理の流れは以下の通りです。

1. pre_process_function の定義を元に、OpenSearch の Ingestion pipeline もしくは Search pipline 内の Text embeddding processor から与えられた入力から、推論エンドポイントに与えるパラメーターを作成
1. pre_process_function によって変換されたパラメーターを元に、request_body の定義に沿ってペイロードを組み立て、推論エンドポイントの呼び出しを行う
1. post_process_function の定義を元に、推論エンドポイントから返却された推論結果を加工し、Text embedding processor に返却

In [26]:
embedding_model_name = "amazon.titan-embed-text-v2:0"

payload = {
  "name": embedding_model_name, 
  "description": "Remote connector for " + embedding_model_name,
  "version": 1, 
  "protocol": "aws_sigv4",
  "credential": {
    "roleArn": opensearch_connector_role_arn
  },
  "parameters": {
    "region": default_region,
    "service_name": "bedrock",
    "model": embedding_model_name,
    "dimensions": 1024,
    "normalize": True,
    "embeddingTypes": ["float"],    
  },
  "actions": [
    {
      "action_type": "predict",
      "method": "POST",
      "headers": {
          "content-type": "application/json",
          "x-amz-content-sha256": "required",
      },
      "url": "https://bedrock-runtime.${parameters.region}.amazonaws.com/model/${parameters.model}/invoke",
      "pre_process_function": "connector.pre_process.bedrock.embedding",
      "request_body": '{ "inputText": "${parameters.inputText}", "dimensions": ${parameters.dimensions}, "normalize": ${parameters.normalize}, "embeddingTypes": ${parameters.embeddingTypes} }',
      "post_process_function": "connector.post_process.bedrock.embedding",
    }
  ]
}

# API の実行
response = opensearch_client.http.post("/_plugins/_ml/connectors/_create", body=payload)

# 結果からコネクタ ID を取得
opensearch_embedding_connector_id = response["connector_id"]
print("embedding connector id: " + opensearch_embedding_connector_id)

embedding connector id: 2e4dbd08-48ea-4b06-9ac7-008db54f201e


##### OpenSearch へのモデル登録
コネクタを元に、OpenSearch にモデル情報を登録します。

In [28]:
payload = {
    "name": embedding_model_name,
    "description": embedding_model_name,
    "function_name": "remote",
    "connector_id": opensearch_embedding_connector_id
}
response = opensearch_client.http.post("/_plugins/_ml/models/_register?deploy=true", body=payload)

opensearch_embedding_model_id = response['model_id']

for i in range(300):
    ml_model_status = opensearch_client.http.get("/_plugins/_ml/models/"+ opensearch_embedding_model_id)
    model_state = ml_model_status.get("model_state")
    if model_state in ["DEPLOYED", "PARTIALLY_DEPLOYED"]:
        break
    time.sleep(1)

if model_state == "DEPLOYED":
    print("embedding model " + opensearch_embedding_model_id + " is deployed successfully")
elif model_state == "PARTIALLY_DEPLOYED":
    print("embedding model " + opensearch_embedding_model_id + " is deployed only partially")
else:
    raise Exception("embedding model " + opensearch_embedding_model_id + " deployment failed")

print(ml_model_status)

embedding model a130deda-1f47-40e5-b2f6-0982f6a45bfb is deployed successfully
{'name': 'amazon.titan-embed-text-v2:0', 'model_group_id': '10fd295f-e0fe-42b4-a9c0-783059987abd', 'algorithm': 'REMOTE', 'model_version': '4', 'description': 'amazon.titan-embed-text-v2:0', 'model_state': 'DEPLOYED', 'created_time': 1756604144517, 'last_updated_time': 1756604144638, 'last_deployed_time': 1756604144638, 'auto_redeploy_retry_times': 0, 'planning_worker_node_count': 1, 'current_worker_node_count': 1, 'planning_worker_nodes': ['-8kLEkILRz-t6t049rIcgQ'], 'deploy_to_all_nodes': True, 'is_hidden': False, 'connector_id': '2e4dbd08-48ea-4b06-9ac7-008db54f201e', 'tenant_id': '720701806597:workshop-vector'}


#### モデルの呼び出しテスト
OpenSearch 経由で Amazon Bedrock 上の埋め込みモデルを実行できることを確認します。<br>
モデルの呼び出し方は 2 パターンあります。

##### Text embedding processor からの呼び出しを想定したテストパターン
Text embedding processor からの呼び出しを想定する場合は、以下パスの API を使用します。<br>
Text embedding processor が text_embedding モデルを呼び出す際のパラメーターキーは text_docs で固定されています。同パラメーターには、クライアントからの入力テキストがセットされています。

<div class="alert alert-block alert-warning"> 
AOSS では、text_embedding 経由の呼び出しはサポートされていません。
</div>

##### pre_process_function をバイパスするパターン
ML モデル配下の predict API を直接呼び出してテストを行うことも可能です。<br>
この場合 pre_process_function は呼び出されず、parameters に記載した値が直接コネクタで指定した推論エンドポイントに渡されます。

In [29]:
path = "/_plugins/_ml/models/" + opensearch_embedding_model_id + "/_predict"
payload = {
  "parameters": {
    "inputText": "日本で梅雨がないのはどこ？"
  }
}
response = opensearch_client.http.post(path, body=payload)
response

{'inference_results': [{'output': [{'name': 'sentence_embedding',
     'data_type': 'FLOAT32',
     'shape': [1024],
     'data': [-0.04390088468790054,
      0.017402151599526405,
      -0.016512269154191017,
      -0.005462891422212124,
      0.046669408679008484,
      0.004152786452323198,
      -0.027685241773724556,
      -0.00731681426987052,
      0.027685241773724556,
      -0.06288505345582962,
      -0.02669648453593254,
      -0.03223353251814842,
      -0.031640276312828064,
      -0.031838029623031616,
      0.02867400087416172,
      0.0034606552217155695,
      -0.027487490326166153,
      -0.010381965897977352,
      -0.007860631681978703,
      0.025509973987936974,
      0.001915719942189753,
      0.04904242977499962,
      -0.04904242977499962,
      -0.04568064957857132,
      -0.024026835337281227,
      -0.027487490326166153,
      -0.04844917356967926,
      -0.05022893846035004,
      -0.0018786414293572307,
      0.0205661803483963,
      0.04172561690211296,

#### Ingestion pipeline の作成
データ登録時にベクトル埋め込みを行う Ingestion pipeline を作成します。埋め込み元のデータはテキストであるため、今回は [Text embedding processor](https://opensearch.org/docs/latest/ingest-pipelines/processors/text-embedding/) を使用します。

Text embedding processor では、埋め込みの元となるフィールドと埋め込みを格納するフィールドのマッピングを field_map 内で定義し、model_id には埋め込みに用いるモデル ID を指定します。

以下は Ingestion pipeline による埋め込みのイメージです

<img src="./img/neural-search-ingestion.png">

In [30]:
payload = {
  "processors": [
    {
      "text_embedding": {
        "model_id": opensearch_embedding_model_id,
        "field_map": {
            "question": "question_embedding",
            "context": "context_embedding"
        }
      }
    }
  ]
}

ingestion_pipeline_id = f"{embedding_model_name}_neural_search_ingestion"

response = opensearch_client.http.put("/_ingest/pipeline/" + ingestion_pipeline_id, body=payload)
print(response)

response = opensearch_client.http.get("/_ingest/pipeline/" + ingestion_pipeline_id)
print(response)

{'acknowledged': True}
{'amazon.titan-embed-text-v2:0_neural_search_ingestion': {'processors': [{'text_embedding': {'model_id': 'af5ca3a0-f68b-4a20-a111-f22a669cefa8', 'field_map': {'question': 'question_embedding', 'context': 'context_embedding'}}}]}}


作成したパイプラインは _simulate API でテストが可能です。<br>
context_embedding および question_embedding フィールドが含まれていれば正常にパイプラインが動作していると判断できます。

<div class="alert alert-block alert-warning"> 
AOSS では、pipeline_id 指定の _simulate API は、使用できませんので、厳密には「作成した」パイプラインのテストはできません。同じ内容を再定義してテストすることは可能です。
</div>

In [31]:
%%time
payload = {
  "docs": [
    {
      "_index": "testindex1",
      "_id": "1",
      "_source":{
         "question": "日本で梅雨がないのはどこか。",
         "context": "梅雨 [SEP] 梅雨（つゆ、ばいう）は、北海道と小笠原諸島を除く日本、朝鮮半島南部、中国の南部から長江流域にかけての沿海部、および台湾など、東アジアの広範囲においてみられる特有の気象現象で、5月から7月にかけて来る曇りや雨の多い期間のこと。雨季の一種である。 ",
      }
    }
  ],
  "pipeline": {
    'processors': [
      {
        'text_embedding': {
          'model_id': opensearch_embedding_model_id, 
          'field_map': {
            'question': 'question_embedding', 
            'context': 'context_embedding'
          }
        }
      }
    ]
  }
}
response = opensearch_client.http.post("/_ingest/pipeline/_simulate", body=payload)
print(response)

{'docs': [{'doc': {'_index': 'testindex1', '_id': '1', '_source': {'context': '梅雨 [SEP] 梅雨（つゆ、ばいう）は、北海道と小笠原諸島を除く日本、朝鮮半島南部、中国の南部から長江流域にかけての沿海部、および台湾など、東アジアの広範囲においてみられる特有の気象現象で、5月から7月にかけて来る曇りや雨の多い期間のこと。雨季の一種である。 ', 'context_embedding': [-0.007301194, 0.06679988, -0.0042758374, -0.009681142, 0.07260856, -0.038724568, -0.020491749, -0.0051229373, 0.024525559, -0.039531328, -0.01984634, -0.04227432, -0.038079157, -0.010810608, 0.05066464, 0.04582407, -0.015247798, -0.01266616, -0.015247798, 0.02597773, -0.01669997, 0.09487519, -0.038885918, -0.01815214, -0.0308183, -0.07680372, -0.01984634, 0.0013916641, -0.015731854, -0.00992317, 0.031786416, 0.010407227, -0.016215913, -0.025493672, 0.0027429902, 0.066154465, -0.020975808, -0.006776799, 0.009882832, -0.00029371172, 0.011536694, -0.014279684, -0.00025337364, 0.09293896, -0.018958902, -0.01879755, 0.020088369, 0.027752606, 0.04162891, -0.01710335, 0.037111044, -0.048083004, -0.0028034972, 0.039208625, -0.00054960646, 0.005929699, -0.00463888

#### Search pipeline の作成
クライアントから入力されたテキストベースのクエリをベクトルベースのクエリに変換するための Search pipeline を作成します。

Search pipeline は、検索時のクエリ書き換え用の Request processors、レスポンス書き換え用の Response processors、スコアなどの検索結果を書き換える Search phase results processors の 3 タイプが存在します。

<img src='./img/search-pipelines.png'>

今回使用する [Neural query enricher processor][neural-query-enricher] は Request processors に属しています。このプロセッサは、後述する Neural query を実行する際のデフォルトモデルをセットするものです。

[search-request-processors]: https://opensearch.org/docs/latest/search-plugins/search-pipelines/search-processors/#search-request-processors
[search-response-processors]: https://opensearch.org/docs/latest/search-plugins/search-pipelines/search-processors/#search-response-processors
[search-phase-results-processors]: https://opensearch.org/docs/latest/search-plugins/search-pipelines/search-processors/#search-phase-results-processors
[neural-query-enricher]: https://opensearch.org/docs/latest/search-plugins/search-pipelines/neural-query-enricher/

In [32]:
payload={
  "request_processors": [
    {
      "neural_query_enricher" : {
        "default_model_id": opensearch_embedding_model_id
      }
    }
  ]
}
# パイプライン ID の指定
search_pipeline_id = f"{embedding_model_name}_neural_search_query"
# パイプライン作成 API の呼び出し
response = opensearch_client.http.put("/_search/pipeline/" + search_pipeline_id, body=payload)
print(response)

response = opensearch_client.http.get("/_search/pipeline/" + search_pipeline_id)
print(response)

{'acknowledged': True}
{'amazon.titan-embed-text-v2:0_neural_search_query': {'request_processors': [{'neural_query_enricher': {'default_model_id': 'af5ca3a0-f68b-4a20-a111-f22a669cefa8'}}]}}


Search pipeline についてはテスト用の API が提供されていないため、実際に Neural search を実行して動作を確認していきます。

## ニューラル検索の実行
データセットを OpenSearch にロードし、検索を実行していきます。

### データロード
DataFrame 形式に変換したサンプルデータセットを OpenSearch に登録していきます。DataFrame にはベクトルデータは含まれていませんが、Ingestion pipeline を通じてデータを登録することでベクトルデータが OpenSearch 側で生成・登録されます。

<div class="alert alert-block alert-warning"> 
AOSS のベクトル検索コレクションでは、Document ID 指定ができません。<br>
指定すると、`Document ID is not supported in create/index operation request` となります。
</div>

<div class="alert alert-block alert-warning"> 
AOSS 側ではなく、Bedrock 側の問題かもしれませんが、シングルスレッドの逐次処置にしないと最後まで登録が完了しません。このため、以下の処理は、15分前後かかります。
</div>

In [33]:
%%time
index_name = "jsquad-neural-search"
response = wr.opensearch.index_df(
    client=opensearch_client,
    df=valid_df,
    use_threads=False,
    index=index_name,
    bulk_size=1, # 1 件ずつ書き込み
    refresh=False,
    pipeline=ingestion_pipeline_id
)

CPU times: user 6.98 s, sys: 370 ms, total: 7.35 s
Wall time: 14min 39s


response["success"] の値が DataFrame の件数と一致しているかを確認します。<br>
True が表示される場合は全件登録に成功していると判断できます。

In [34]:
response["success"] == valid_df["id"].count()

np.True_

パイプラインを通して登録されたドキュメントにベクトルデータが登録されていることを確認します。<br>
**_source.question_embedding** および **_source.context_embedding** フィールドに数値配列が格納されていれば、パイプラインによる埋め込み生成とベクトルデータの格納が正常に行われたと判断することができます。

In [35]:
%%time
index_name = "jsquad-neural-search"
payload = {
  "size": 10,
  "query": {
    "match_all": {}
  },
}
# 検索 API を実行
response = opensearch_client.search(
    body = payload,
    index = index_name,
    filter_path = "hits.hits"
)

# 結果を表示
pd.json_normalize(response["hits"]["hits"])

CPU times: user 9.12 ms, sys: 148 μs, total: 9.27 ms
Wall time: 121 ms


,_index,_id,_score,_source.question,_source.question_embedding,_source.context,_source.answers,_source.id,_source.context_embedding
0,jsquad-neural-search,1%3A0%3Au3_f_ZgBGnBxNeQdtH5H,1.0,電力小売の分野の自由化はいつ完了した？,"[-0.035534795, 0.0022507359, 0.0008011725, 0.0...",オランダ [SEP] 電力小売の分野では、1998年から段階的に始まった自由化が2007年で...,[2007年],a1698820p41q0,"[-0.08347909, -0.021137334, -0.00379045, 0.017..."
1,jsquad-neural-search,1%3A0%3Af9zf_ZgBQNa1Y6dutgaL,1.0,オランダでの電力小売は、1998年から段階的に始まったが、何年に完了したか？,"[-0.074592926, -0.02111638, -0.014603211, 0.05...",オランダ [SEP] 電力小売の分野では、1998年から段階的に始まった自由化が2007年で...,[2007年],a1698820p41q3,"[-0.08347909, -0.021137334, -0.00379045, 0.017..."
2,jsquad-neural-search,1%3A0%3AvX_f_ZgBGnBxNeQdt35G,1.0,1998年から段階的に始まった電力小売りの自由化が完了したのはいつ？,"[-0.036499426, -0.00048643534, 0.0067765475, 0...",オランダ [SEP] 電力小売の分野では、1998年から段階的に始まった自由化が2007年で...,[2007年],a1698820p41q4,"[-0.08347909, -0.021137334, -0.00379045, 0.017..."
3,jsquad-neural-search,1%3A0%3Avn_f_ZgBGnBxNeQduH7O,1.0,オランダの工業化はなぜ遅れた？,"[-0.065249085, 0.01512396, -0.026682986, 0.049...",オランダ [SEP] オランダの工業化は天然資源の欠乏のために遅れはしたが、19世紀半ば以後...,"[天然資源の欠乏, 天然資源の欠乏のため]",a1698820p42q1,"[-0.057039425, 0.023015907, -0.0052953265, 0.0..."
4,jsquad-neural-search,1%3A0%3Agdzf_ZgBQNa1Y6duuQaF,1.0,オランダの工業化ぎ遅れたのはなぜか？,"[-0.06306191, 0.016096221, -0.02712103, 0.0604...",オランダ [SEP] オランダの工業化は天然資源の欠乏のために遅れはしたが、19世紀半ば以後...,"[天然資源の欠乏, 天然資源の欠乏のため]",a1698820p42q2,"[-0.057039425, 0.023015907, -0.0052953265, 0.0..."
5,jsquad-neural-search,1%3A0%3Agtzf_ZgBQNa1Y6duuwYE,1.0,オランダは２０世紀に入って何を確立した？,"[-0.103274934, -0.018473748, -0.0042567523, 0....",オランダ [SEP] オランダは鉱物資源が乏しいにもかかわらず、20世紀に入って金属工業を確...,[金属工業],a1698820p43q0,"[-0.04601226, 0.048057247, -0.0042816964, 0.00..."
6,jsquad-neural-search,1%3A0%3AwH_f_ZgBGnBxNeQdu37R,1.0,オランダでは近年、船舶の建造から他の業種へ切り替えの傾向にあるが、それは何か？,"[-0.040722925, 0.02965138, -0.025706345, 0.019...",オランダ [SEP] オランダは鉱物資源が乏しいにもかかわらず、20世紀に入って金属工業を確...,[石油化学工場の建設工事],a1698820p43q1,"[-0.04601226, 0.048057247, -0.0042816964, 0.00..."
7,jsquad-neural-search,1%3A0%3Ag9zf_ZgBQNa1Y6duvAZ3,1.0,鉱物資源が乏しいにもかかわらず、20世紀に入って金属工業を確立した国の名は？,"[-0.070217274, 0.063141115, 0.0014628599, 0.04...",オランダ [SEP] オランダは鉱物資源が乏しいにもかかわらず、20世紀に入って金属工業を確...,[オランダ],a1698820p43q2,"[-0.04601226, 0.048057247, -0.0042816964, 0.00..."
8,jsquad-neural-search,1%3A0%3AwX_f_ZgBGnBxNeQdvX5A,1.0,オランダで金属工業を確立したのはいつ？,"[-0.06193185, 0.029684577, -0.014308393, 0.016...",オランダ [SEP] オランダは鉱物資源が乏しいにもかかわらず、20世紀に入って金属工業を確...,[20世紀],a1698820p43q3,"[-0.04601226, 0.048057247, -0.0042816964, 0.00..."
9,jsquad-neural-search,1%3A0%3AhNzf_ZgBQNa1Y6duvQbw,1.0,オランダは金属工業が盛んだが具体的には？,"[-0.057787415, 0.06135454, 0.0093934275, 0.017...",オランダ [SEP] 代表的な製造業としては製鉄、機械、電気機器、造船、航空機などの金属工業...,"[製鉄、機械、電気機器、造船、航空機, 製鉄、機械、電気機器、造船、航空機など]",a1698820p44q0,"[-0.07574531, 0.023649735, -0.0065326192, 0.00..."


### Neural query によるニューラル検索の実行
[Neural query][neural] を使うことで、Search pipeline から埋め込みモデルを呼び出し、ユーザーのクエリデータを OpenSearch 側でベクトルデータに変換してから内部的にベクトル検索を実行することが可能となります。

仕組みは以下の通りです。model_id で指定した埋め込みモデルを使用し、query_text パラメーターに入力されたクエリテキストから生成したベクトルで knn query を実行しています。

<img src="./img/neural-search-query.png">

以下は question フィールドから生成された question_dense_embedding フィールドに対する Neural query の実行サンプルです。

[neural]: https://opensearch.org/docs/latest/query-dsl/specialized/neural/

In [36]:
%%time
index_name = "jsquad-neural-search"
query = "日本で梅雨がない場所は？"
payload = {
  "size": 10,
  "query": {
    "neural": {
      "question_embedding": {
        "query_text": query, 
        "model_id": opensearch_embedding_model_id,
        "k": 10
      }
    }
  },
  "_source" : False,
  "fields": ["question", "answers",  "context"]
}
# 検索 API を実行
response = opensearch_client.search(
    body = payload,
    index = index_name,
    filter_path = "hits.hits"
)

# 結果を表示
pd.json_normalize(response["hits"]["hits"])

CPU times: user 4.09 ms, sys: 0 ns, total: 4.09 ms
Wall time: 313 ms


,_index,_id,_score,fields.question,fields.answers,fields.context
0,jsquad-neural-search,1%3A0%3AtNzZ_ZgBQNa1Y6du3ALV,0.832107,[梅雨が日本の中でない地域はどこか。],"[北海道, 東北地方]",[梅雨 [SEP] 年によっては梅雨明けの時期が特定できなかったり、あるいは発表がされないこ...
1,jsquad-neural-search,1%3A0%3AktzZ_ZgBQNa1Y6duogLg,0.828650,[日本で梅雨がないのは北海道とどこか。],"[小笠原諸島, 小笠原諸島を除く日本]",[梅雨 [SEP] 梅雨（つゆ、ばいう）は、北海道と小笠原諸島を除く日本、朝鮮半島南部、中国...
2,jsquad-neural-search,1%3A0%3AxNzZ_ZgBQNa1Y6du9gIh,0.659169,[気候学的には梅雨はないとされている場所は？],[北海道],[梅雨 [SEP] 実際の気象としては北海道にも道南を中心に梅雨前線がかかることはあるが、平...
3,jsquad-neural-search,1%3A0%3AAn_Z_ZgBGnBxNeQd9nv7,0.658166,[梅雨がないとされている都道府県はどこ？],[北海道],[梅雨 [SEP] 実際の気象としては北海道にも道南を中心に梅雨前線がかかることはあるが、平...
4,jsquad-neural-search,1%3A0%3AqNzZ_ZgBQNa1Y6duygIE,0.567400,[日本の地域で本格的な長雨に突入しない場所はどこか。],[北海道],[梅雨 [SEP] 次に梅雨前線は中国の江淮（長江流域・淮河流域）に北上する。6月下旬には華...
5,jsquad-neural-search,1%3A0%3AF3_a_ZgBGnBxNeQdGHsc,0.550389,[ほとんど雨が降らない梅雨を何と呼ぶか],"[空梅雨, 空梅雨（からつゆ）]",[梅雨 [SEP] 梅雨の期間中ほとんど雨が降らない場合がある。このような梅雨のことを空梅雨...
6,jsquad-neural-search,1%3A0%3A2tza_ZgBQNa1Y6duGALS,0.541300,[ほとんど雨が降らない梅雨を何という？],"[空梅雨, 空梅雨（からつゆ）]",[梅雨 [SEP] 梅雨の期間中ほとんど雨が降らない場合がある。このような梅雨のことを空梅雨...
7,jsquad-neural-search,1%3A0%3As9zZ_ZgBQNa1Y6du2wJE,0.537503,[年によっては梅雨入りの時期が特定できなかったり、あるいは発表がされないこともある。東・西日...,"[数年に一回, 数年に一回の割合, 数年に一回の割合で起こる]",[梅雨 [SEP] 年によっては梅雨入りの時期が特定できなかったり、あるいは発表がされないこ...
8,jsquad-neural-search,1%3A0%3A_3_Z_ZgBGnBxNeQd8npW,0.529294,[日本付近の梅雨期の雨量が最も多い地区は],[九州南部],[梅雨 [SEP] 梅雨の期間はどの地方でも40日から50日前後と大差はないが、期間中の降水...
9,jsquad-neural-search,1%3A0%3A9n_Z_ZgBGnBxNeQd5HpE,0.527282,[気象学では一般的に、梅雨がある中国沿海部・朝鮮半島・日本列島の大部分を何に含めるか],"[モンスーン・アジア, モンスーン・アジアに含める。]",[梅雨 [SEP] アジア・モンスーンの影響範囲はさらに東にまで及んでおり、南シナ海を覆う熱...


Neural query を使用することで、クライアントはテキストとモデル ID を渡すだけで、裏でベクトル検索が実行されるようになりました。

ただ、クライアントがモデル ID を検索の都度指定するのは不便に思えます。

そこで、先ほどの Search pipeline を使用して、クライアントがモデル ID を指定せずに Neural query を実行できるようにします。

In [37]:
%%time
index_name = "jsquad-neural-search"
query = "日本で梅雨がない場所は？"
payload = {
  "size": 10,
  "query": {
    "neural": {
      "question_embedding": {
        "query_text": query, 
        # model_id の指定は行わない
        "k": 10
      }
    }
  },
  "_source" : False,
  "fields": ["question", "answers",  "context"]
}
# 検索 API を実行
response = opensearch_client.search(
    body = payload,
    index = index_name,
    filter_path = "hits.hits",
    search_pipeline = search_pipeline_id # 新たに追加
)

# 結果を表示
pd.json_normalize(response["hits"]["hits"])

CPU times: user 4.11 ms, sys: 0 ns, total: 4.11 ms
Wall time: 261 ms


,_index,_id,_score,fields.question,fields.answers,fields.context
0,jsquad-neural-search,1%3A0%3AtNzZ_ZgBQNa1Y6du3ALV,0.832107,[梅雨が日本の中でない地域はどこか。],"[北海道, 東北地方]",[梅雨 [SEP] 年によっては梅雨明けの時期が特定できなかったり、あるいは発表がされないこ...
1,jsquad-neural-search,1%3A0%3AktzZ_ZgBQNa1Y6duogLg,0.828650,[日本で梅雨がないのは北海道とどこか。],"[小笠原諸島, 小笠原諸島を除く日本]",[梅雨 [SEP] 梅雨（つゆ、ばいう）は、北海道と小笠原諸島を除く日本、朝鮮半島南部、中国...
2,jsquad-neural-search,1%3A0%3AxNzZ_ZgBQNa1Y6du9gIh,0.659169,[気候学的には梅雨はないとされている場所は？],[北海道],[梅雨 [SEP] 実際の気象としては北海道にも道南を中心に梅雨前線がかかることはあるが、平...
3,jsquad-neural-search,1%3A0%3AAn_Z_ZgBGnBxNeQd9nv7,0.658166,[梅雨がないとされている都道府県はどこ？],[北海道],[梅雨 [SEP] 実際の気象としては北海道にも道南を中心に梅雨前線がかかることはあるが、平...
4,jsquad-neural-search,1%3A0%3AqNzZ_ZgBQNa1Y6duygIE,0.567400,[日本の地域で本格的な長雨に突入しない場所はどこか。],[北海道],[梅雨 [SEP] 次に梅雨前線は中国の江淮（長江流域・淮河流域）に北上する。6月下旬には華...
5,jsquad-neural-search,1%3A0%3AF3_a_ZgBGnBxNeQdGHsc,0.550389,[ほとんど雨が降らない梅雨を何と呼ぶか],"[空梅雨, 空梅雨（からつゆ）]",[梅雨 [SEP] 梅雨の期間中ほとんど雨が降らない場合がある。このような梅雨のことを空梅雨...
6,jsquad-neural-search,1%3A0%3A2tza_ZgBQNa1Y6duGALS,0.541300,[ほとんど雨が降らない梅雨を何という？],"[空梅雨, 空梅雨（からつゆ）]",[梅雨 [SEP] 梅雨の期間中ほとんど雨が降らない場合がある。このような梅雨のことを空梅雨...
7,jsquad-neural-search,1%3A0%3As9zZ_ZgBQNa1Y6du2wJE,0.537503,[年によっては梅雨入りの時期が特定できなかったり、あるいは発表がされないこともある。東・西日...,"[数年に一回, 数年に一回の割合, 数年に一回の割合で起こる]",[梅雨 [SEP] 年によっては梅雨入りの時期が特定できなかったり、あるいは発表がされないこ...
8,jsquad-neural-search,1%3A0%3A_3_Z_ZgBGnBxNeQd8npW,0.529294,[日本付近の梅雨期の雨量が最も多い地区は],[九州南部],[梅雨 [SEP] 梅雨の期間はどの地方でも40日から50日前後と大差はないが、期間中の降水...
9,jsquad-neural-search,1%3A0%3A9n_Z_ZgBGnBxNeQd5HpE,0.527282,[気象学では一般的に、梅雨がある中国沿海部・朝鮮半島・日本列島の大部分を何に含めるか],"[モンスーン・アジア, モンスーン・アジアに含める。]",[梅雨 [SEP] アジア・モンスーンの影響範囲はさらに東にまで及んでおり、南シナ海を覆う熱...


### 距離とスコアによるフィルタリング
[ベクトル検索の実装 (Amazon Bedrock 編)](#vector-search-with-sagemaker) でも解説した、min_score および max_distance によるフィルタリングは Neural query でも利用可能です。

In [38]:
%%time
index_name = "jsquad-neural-search"
query = "日本で梅雨がない場所は？"
payload = {
  "size": 10,
  "query": {
    "neural": {
      "question_embedding": {
        "query_text": query, 
        "min_score": 0.7
      }
    }
  },
  "_source" : False,
  "fields": ["question", "answers",  "context"]
}
# 検索 API を実行
response = opensearch_client.search(
    body = payload,
    index = index_name,
    filter_path = "hits.hits",
    search_pipeline = search_pipeline_id
)

# 結果を表示
pd.json_normalize(response["hits"]["hits"])

CPU times: user 3.99 ms, sys: 0 ns, total: 3.99 ms
Wall time: 234 ms


,_index,_id,_score,fields.question,fields.answers,fields.context
0,jsquad-neural-search,1%3A0%3AtNzZ_ZgBQNa1Y6du3ALV,0.832107,[梅雨が日本の中でない地域はどこか。],"[北海道, 東北地方]",[梅雨 [SEP] 年によっては梅雨明けの時期が特定できなかったり、あるいは発表がされないこ...
1,jsquad-neural-search,1%3A0%3AktzZ_ZgBQNa1Y6duogLg,0.828650,[日本で梅雨がないのは北海道とどこか。],"[小笠原諸島, 小笠原諸島を除く日本]",[梅雨 [SEP] 梅雨（つゆ、ばいう）は、北海道と小笠原諸島を除く日本、朝鮮半島南部、中国...


In [39]:
%%time
index_name = "jsquad-neural-search"
query = "日本で梅雨がない場所は？"
payload = {
  "size": 10,
  "query": {
    "neural": {
      "question_embedding": {
        "query_text": query, 
        "max_distance": 0.3
      }
    }
  },
  "_source" : False,
  "fields": ["question", "answers",  "context"]
}
# 検索 API を実行
response = opensearch_client.search(
    body = payload,
    index = index_name,
    filter_path = "hits.hits",
    search_pipeline = search_pipeline_id
)

# 結果を表示
pd.json_normalize(response["hits"]["hits"])

CPU times: user 3.78 ms, sys: 61 μs, total: 3.84 ms
Wall time: 133 ms


,_index,_id,_score,fields.question,fields.answers,fields.context
0,jsquad-neural-search,1%3A0%3AtNzZ_ZgBQNa1Y6du3ALV,0.832107,[梅雨が日本の中でない地域はどこか。],"[北海道, 東北地方]",[梅雨 [SEP] 年によっては梅雨明けの時期が特定できなかったり、あるいは発表がされないこ...
1,jsquad-neural-search,1%3A0%3AktzZ_ZgBQNa1Y6duogLg,0.828650,[日本で梅雨がないのは北海道とどこか。],"[小笠原諸島, 小笠原諸島を除く日本]",[梅雨 [SEP] 梅雨（つゆ、ばいう）は、北海道と小笠原諸島を除く日本、朝鮮半島南部、中国...


### Appendix: ニューラル検索と従来のベクトル検索を組み合わせて使用する
本ラボでは、ニューラル検索を Ingest pipeline による登録データのテキスト(または画像)→ベクトル変換と、Search pipeline による検索クエリのベクトル変換を組み合わせて実装していきました。

Ingest pipeline と Search pipeline の併用は、実は必須ではありません。以下のようにどちらか一方のみを使用することもできます。
- データ登録は Ingest pipeline を通じて行うが、検索ではバックエンド側でベクトル生成を行ったうえで通常の knn query を用いる
- データ登録はバックエンド側でベクトル生成を行ったうえで、Ingest pipeline を使わず通常の bulk API で登録する。検索でのみ Search pipeline を介した neural query を用いる

例えば、以下のようなユースケースが考えられます。

- 初期構築時に極めて大量のベクトル登録を行う必要があるため、バッチ推論を使用して非同期でベクトルデータを生成し、通常の bulk API で登録を実施。初期構築後の部分更新、検索は Neural query で行う

以下のサンプルコードでは、[ベクトル検索の実装 (Amazon Bedrock 編)](#vector-search-with-sagemaker) で作成した kNN 検索用のインデックスに対して Neural query を実行しています。

In [40]:
%%time
index_name = "jsquad-knn"
query = "日本で梅雨がない場所は？"
payload = {
  "size": 10,
  "query": {
    "neural": {
      "question_embedding": {
        "query_text": query, 
        "k": 10
      }
    }
  },
  "_source" : False,
  "fields": ["question", "answers",  "context"]
}
# 検索 API を実行
response = opensearch_client.search(
    body = payload,
    index = index_name,
    filter_path = "hits.hits",
    search_pipeline = search_pipeline_id
)

# 結果を表示
pd.json_normalize(response["hits"]["hits"])

CPU times: user 4.14 ms, sys: 0 ns, total: 4.14 ms
Wall time: 242 ms


,_index,_id,_score,fields.question,fields.answers,fields.context
0,jsquad-knn,1%3A0%3AFNuQ_ZgBQNa1Y6duMPp-,0.832107,[梅雨が日本の中でない地域はどこか。],"[北海道, 東北地方]",[梅雨 [SEP] 年によっては梅雨明けの時期が特定できなかったり、あるいは発表がされないこ...
1,jsquad-knn,1%3A0%3A0NuQ_ZgBQNa1Y6duMPl-,0.828650,[日本で梅雨がないのは北海道とどこか。],"[小笠原諸島, 小笠原諸島を除く日本]",[梅雨 [SEP] 梅雨（つゆ、ばいう）は、北海道と小笠原諸島を除く日本、朝鮮半島南部、中国...
2,jsquad-knn,1%3A0%3ANNuQ_ZgBQNa1Y6duMPp-,0.659169,[気候学的には梅雨はないとされている場所は？],[北海道],[梅雨 [SEP] 実際の気象としては北海道にも道南を中心に梅雨前線がかかることはあるが、平...
3,jsquad-knn,1%3A0%3ANduQ_ZgBQNa1Y6duMPp-,0.658166,[梅雨がないとされている都道府県はどこ？],[北海道],[梅雨 [SEP] 実際の気象としては北海道にも道南を中心に梅雨前線がかかることはあるが、平...
4,jsquad-knn,1%3A0%3A_NuQ_ZgBQNa1Y6duMPl-,0.567400,[日本の地域で本格的な長雨に突入しない場所はどこか。],[北海道],[梅雨 [SEP] 次に梅雨前線は中国の江淮（長江流域・淮河流域）に北上する。6月下旬には華...
5,jsquad-knn,1%3A0%3AX9uQ_ZgBQNa1Y6duMPp-,0.550389,[ほとんど雨が降らない梅雨を何と呼ぶか],"[空梅雨, 空梅雨（からつゆ）]",[梅雨 [SEP] 梅雨の期間中ほとんど雨が降らない場合がある。このような梅雨のことを空梅雨...
6,jsquad-knn,1%3A0%3AYNuQ_ZgBQNa1Y6duMPp-,0.541300,[ほとんど雨が降らない梅雨を何という？],"[空梅雨, 空梅雨（からつゆ）]",[梅雨 [SEP] 梅雨の期間中ほとんど雨が降らない場合がある。このような梅雨のことを空梅雨...
7,jsquad-knn,1%3A0%3AEtuQ_ZgBQNa1Y6duMPp-,0.537503,[年によっては梅雨入りの時期が特定できなかったり、あるいは発表がされないこともある。東・西日...,"[数年に一回, 数年に一回の割合, 数年に一回の割合で起こる]",[梅雨 [SEP] 年によっては梅雨入りの時期が特定できなかったり、あるいは発表がされないこ...
8,jsquad-knn,1%3A0%3AL9uQ_ZgBQNa1Y6duMPp-,0.529294,[日本付近の梅雨期の雨量が最も多い地区は],[九州南部],[梅雨 [SEP] 梅雨の期間はどの地方でも40日から50日前後と大差はないが、期間中の降水...
9,jsquad-knn,1%3A0%3AHduQ_ZgBQNa1Y6duMPp-,0.527282,[気象学では一般的に、梅雨がある中国沿海部・朝鮮半島・日本列島の大部分を何に含めるか],"[モンスーン・アジア, モンスーン・アジアに含める。]",[梅雨 [SEP] アジア・モンスーンの影響範囲はさらに東にまで及んでおり、南シナ海を覆う熱...


また、本ラボで構築した Neural search 用のインデックスに対して、knn query を実行することもできます。以下はサンプルコードです。


In [41]:
index_name = "jsquad-neural-search"
query = "日本で梅雨がない場所は？"

def text_to_embedding(text, region_name, model_id):
    payload = {
        "inputText": text
    }
    body = json.dumps(payload)
    bedrock_runtime_client = boto3.client("bedrock-runtime", region_name)
    response = bedrock_runtime_client.invoke_model(modelId = model_id, body=body)
    model_response = json.loads(response["body"].read())
    return model_response["embedding"]


vector = text_to_embedding(text=query, region_name=default_region, model_id=model_id)
k = 10

payload = {
  "query": {
    "knn": {
      "question_embedding": {
        "vector": vector,
        "k": k
      }
    }
  },
  "_source": False,
  "fields": ["question", "answers", "context"],
  "size": k
}
response = opensearch_client.search(
    index=index_name,
    body=payload
)
pd.json_normalize(response["hits"]["hits"])

,_index,_id,_score,fields.question,fields.answers,fields.context
0,jsquad-neural-search,1%3A0%3AtNzZ_ZgBQNa1Y6du3ALV,0.832107,[梅雨が日本の中でない地域はどこか。],"[北海道, 東北地方]",[梅雨 [SEP] 年によっては梅雨明けの時期が特定できなかったり、あるいは発表がされないこ...
1,jsquad-neural-search,1%3A0%3AktzZ_ZgBQNa1Y6duogLg,0.828650,[日本で梅雨がないのは北海道とどこか。],"[小笠原諸島, 小笠原諸島を除く日本]",[梅雨 [SEP] 梅雨（つゆ、ばいう）は、北海道と小笠原諸島を除く日本、朝鮮半島南部、中国...
2,jsquad-neural-search,1%3A0%3AxNzZ_ZgBQNa1Y6du9gIh,0.659169,[気候学的には梅雨はないとされている場所は？],[北海道],[梅雨 [SEP] 実際の気象としては北海道にも道南を中心に梅雨前線がかかることはあるが、平...
3,jsquad-neural-search,1%3A0%3AAn_Z_ZgBGnBxNeQd9nv7,0.658166,[梅雨がないとされている都道府県はどこ？],[北海道],[梅雨 [SEP] 実際の気象としては北海道にも道南を中心に梅雨前線がかかることはあるが、平...
4,jsquad-neural-search,1%3A0%3AqNzZ_ZgBQNa1Y6duygIE,0.567400,[日本の地域で本格的な長雨に突入しない場所はどこか。],[北海道],[梅雨 [SEP] 次に梅雨前線は中国の江淮（長江流域・淮河流域）に北上する。6月下旬には華...
5,jsquad-neural-search,1%3A0%3AF3_a_ZgBGnBxNeQdGHsc,0.550389,[ほとんど雨が降らない梅雨を何と呼ぶか],"[空梅雨, 空梅雨（からつゆ）]",[梅雨 [SEP] 梅雨の期間中ほとんど雨が降らない場合がある。このような梅雨のことを空梅雨...
6,jsquad-neural-search,1%3A0%3A2tza_ZgBQNa1Y6duGALS,0.541300,[ほとんど雨が降らない梅雨を何という？],"[空梅雨, 空梅雨（からつゆ）]",[梅雨 [SEP] 梅雨の期間中ほとんど雨が降らない場合がある。このような梅雨のことを空梅雨...
7,jsquad-neural-search,1%3A0%3As9zZ_ZgBQNa1Y6du2wJE,0.537503,[年によっては梅雨入りの時期が特定できなかったり、あるいは発表がされないこともある。東・西日...,"[数年に一回, 数年に一回の割合, 数年に一回の割合で起こる]",[梅雨 [SEP] 年によっては梅雨入りの時期が特定できなかったり、あるいは発表がされないこ...
8,jsquad-neural-search,1%3A0%3A_3_Z_ZgBGnBxNeQd8npW,0.529294,[日本付近の梅雨期の雨量が最も多い地区は],[九州南部],[梅雨 [SEP] 梅雨の期間はどの地方でも40日から50日前後と大差はないが、期間中の降水...
9,jsquad-neural-search,1%3A0%3A9n_Z_ZgBGnBxNeQd5HpE,0.527282,[気象学では一般的に、梅雨がある中国沿海部・朝鮮半島・日本列島の大部分を何に含めるか],"[モンスーン・アジア, モンスーン・アジアに含める。]",[梅雨 [SEP] アジア・モンスーンの影響範囲はさらに東にまで及んでおり、南シナ海を覆う熱...


## まとめ
ラボを通して、OpenSearch 側でベクトル変換を行うニューラル検索の機能を確認できました。時間がある方は、続いて以下のラボも実施してみましょう。

- ~~[スパース検索の実装 (Amazon SageMaker 編)](../sparse-search/sparse-search-with-sagemaker.ipynb)~~

<div class="alert alert-block alert-warning"> 
東京リージョンの Bedrock では、スパース検索が可能なモデルは使用できないので、保留にしました。
</div>

## 後片付け

### データセット削除
ダウンロードしたデータセットを削除します。./dataset ディレクトリ配下に何もない場合は、./dataset ディレクトリも合わせて削除します。

In [42]:
%rm -rf {dataset_dir}

In [43]:
%rmdir ./dataset

rmdir: failed to remove './dataset': No such file or directory


# ハイブリッド検索 (Amazon Bedrock 編)

## 概要
本ラボでは、テキスト検索、ベクトル検索を組み合わせたハイブリッド検索を実装していきます。

<div class="alert alert-block alert-warning"> 
東京リージョンの Bedrock では、スパース検索が可能なモデルは使用できないので、スパース検索は除外しました。
</div>

### ハイブリッド検索の概要
ハイブリッド検索は、複数の検索を実行し、各結果をマージ、スコアを平準化したうえでランク付けを行う機能です。

OpenSearch では [Hybrid query][hybrid-search] と [Normalization processor][normalization-processor] を組み合わせることでハイブリッド検索を実装することができます。

Hybrid query は複数のクエリを実行した結果を組み合わせるものです。単に Hybrid query を実行するだけでは、個々のクエリごとのスコア計算方法やベースのスコア値が大きく異なることで偏った結果となるため、Normalization processor によりスコアの平準化を行います。

以下はハイブリッド検索の処理フローです

<img src="./img/hybrid-search-overview.png" width="1024">

[hybrid-search]: https://opensearch.org/docs/latest/search-plugins/hybrid-search/
[normalization-processor]: https://opensearch.org/docs/latest/search-plugins/search-pipelines/normalization-processor/

## 事前作業
特になし

## ハイブリッド検索の実行

### テキスト検索をベクトル検索で補う
テキスト検索はクエリに厳密にマッチするドキュメントを取得可能です。<br>
一方でクエリに厳密にマッチしないものの、意図としては近いドキュメントまでは拾うことができません。



In [44]:
index_name = "jsquad-neural-search"
query = "日本で梅雨がない地域は？"

payload = {
  "query": {
    "match": {
      "question": {
        "query": query,
        "operator": "and"
      }
    }
  },
  "_source": False,
  "fields": ["question", "answers", "context"],
  "size": 10
}
response = opensearch_client.search(
    index=index_name,
    body=payload
)
pd.json_normalize(response["hits"]["hits"])

""


他方、ベクトル検索は意味的に近いドキュメントを検索することに長けています

In [45]:
# ニューラル検索用のパイプラインで代用する
hybrid_search_pipeline_id = "amazon.titan-embed-text-v2:0_neural_search_query"

index_name = "jsquad-neural-search"
query = "日本で梅雨がない地域は？"

payload = {
  "size": 10,
  "query": {
    "neural": {
      "question_embedding": {
        "query_text": query, 
        "k": 10
      }
    }
  },
  "_source" : False,
  "fields": ["question", "answers",  "context"]
}
response = opensearch_client.search(
    index=index_name,
    body=payload,
    search_pipeline = hybrid_search_pipeline_id 
)
pd.json_normalize(response["hits"]["hits"])

,_index,_id,_score,fields.question,fields.answers,fields.context
0,jsquad-neural-search,1%3A0%3AtNzZ_ZgBQNa1Y6du3ALV,0.868243,[梅雨が日本の中でない地域はどこか。],"[北海道, 東北地方]",[梅雨 [SEP] 年によっては梅雨明けの時期が特定できなかったり、あるいは発表がされないこ...
1,jsquad-neural-search,1%3A0%3AktzZ_ZgBQNa1Y6duogLg,0.823396,[日本で梅雨がないのは北海道とどこか。],"[小笠原諸島, 小笠原諸島を除く日本]",[梅雨 [SEP] 梅雨（つゆ、ばいう）は、北海道と小笠原諸島を除く日本、朝鮮半島南部、中国...
2,jsquad-neural-search,1%3A0%3AAn_Z_ZgBGnBxNeQd9nv7,0.656142,[梅雨がないとされている都道府県はどこ？],[北海道],[梅雨 [SEP] 実際の気象としては北海道にも道南を中心に梅雨前線がかかることはあるが、平...
3,jsquad-neural-search,1%3A0%3AxNzZ_ZgBQNa1Y6du9gIh,0.636189,[気候学的には梅雨はないとされている場所は？],[北海道],[梅雨 [SEP] 実際の気象としては北海道にも道南を中心に梅雨前線がかかることはあるが、平...
4,jsquad-neural-search,1%3A0%3AqNzZ_ZgBQNa1Y6duygIE,0.589473,[日本の地域で本格的な長雨に突入しない場所はどこか。],[北海道],[梅雨 [SEP] 次に梅雨前線は中国の江淮（長江流域・淮河流域）に北上する。6月下旬には華...
5,jsquad-neural-search,1%3A0%3A9n_Z_ZgBGnBxNeQd5HpE,0.541134,[気象学では一般的に、梅雨がある中国沿海部・朝鮮半島・日本列島の大部分を何に含めるか],"[モンスーン・アジア, モンスーン・アジアに含める。]",[梅雨 [SEP] アジア・モンスーンの影響範囲はさらに東にまで及んでおり、南シナ海を覆う熱...
6,jsquad-neural-search,1%3A0%3A_3_Z_ZgBGnBxNeQd8npW,0.537111,[日本付近の梅雨期の雨量が最も多い地区は],[九州南部],[梅雨 [SEP] 梅雨の期間はどの地方でも40日から50日前後と大差はないが、期間中の降水...
7,jsquad-neural-search,1%3A0%3AF3_a_ZgBGnBxNeQdGHsc,0.528213,[ほとんど雨が降らない梅雨を何と呼ぶか],"[空梅雨, 空梅雨（からつゆ）]",[梅雨 [SEP] 梅雨の期間中ほとんど雨が降らない場合がある。このような梅雨のことを空梅雨...
8,jsquad-neural-search,1%3A0%3A2tza_ZgBQNa1Y6duGALS,0.517981,[ほとんど雨が降らない梅雨を何という？],"[空梅雨, 空梅雨（からつゆ）]",[梅雨 [SEP] 梅雨の期間中ほとんど雨が降らない場合がある。このような梅雨のことを空梅雨...
9,jsquad-neural-search,1%3A0%3As9zZ_ZgBQNa1Y6du2wJE,0.516498,[年によっては梅雨入りの時期が特定できなかったり、あるいは発表がされないこともある。東・西日...,"[数年に一回, 数年に一回の割合, 数年に一回の割合で起こる]",[梅雨 [SEP] 年によっては梅雨入りの時期が特定できなかったり、あるいは発表がされないこ...


両者を組み合わせることで、意味的に近い検索結果もフォローしつつ、テキスト検索でマッチするドキュメントについてはよりスコアを上げる = 上位にランク付けすることが可能となります。

In [46]:
%%time
index_name = "jsquad-neural-search"
query = "日本で梅雨がない地域は？"

payload = {
  "size": 10,
  "query": {
    "hybrid": {
      "queries": [
        {
          "match": {
            "question": {
              "query": query,
              "operator": "and"
            }
          }
        },
        {
          "neural": {
            "question_embedding": {
              "query_text": query, # テキストをベクトルに変換し
              "k": 10 # クエリベクトルに近いベクトルのうち上位 10 件を返却
            }
          }
        }
      ]
    }
  },
  "_source" : False,
  "fields": ["question", "answers",  "context"]
}
# 検索 API を実行
response = opensearch_client.search(
    body = payload,
    index = index_name,
    filter_path = "hits.hits",
    search_pipeline = hybrid_search_pipeline_id 
)

# 結果を表示
pd.json_normalize(response["hits"]["hits"])

CPU times: user 4 ms, sys: 65 μs, total: 4.06 ms
Wall time: 386 ms


,_index,_id,_score,fields.question,fields.answers,fields.context
0,jsquad-neural-search,1%3A0%3AktzZ_ZgBQNa1Y6duogLg,-9.549512e+09,[日本で梅雨がないのは北海道とどこか。],"[小笠原諸島, 小笠原諸島を除く日本]",[梅雨 [SEP] 梅雨（つゆ、ばいう）は、北海道と小笠原諸島を除く日本、朝鮮半島南部、中国...
1,jsquad-neural-search,1%3A0%3AktzZ_ZgBQNa1Y6duogLg,-4.422440e+09,[日本で梅雨がないのは北海道とどこか。],"[小笠原諸島, 小笠原諸島を除く日本]",[梅雨 [SEP] 梅雨（つゆ、ばいう）は、北海道と小笠原諸島を除く日本、朝鮮半島南部、中国...
2,jsquad-neural-search,1%3A0%3AktzZ_ZgBQNa1Y6duogLg,-4.422440e+09,[日本で梅雨がないのは北海道とどこか。],"[小笠原諸島, 小笠原諸島を除く日本]",[梅雨 [SEP] 梅雨（つゆ、ばいう）は、北海道と小笠原諸島を除く日本、朝鮮半島南部、中国...
3,jsquad-neural-search,1%3A0%3AktzZ_ZgBQNa1Y6duogLg,8.233960e-01,[日本で梅雨がないのは北海道とどこか。],"[小笠原諸島, 小笠原諸島を除く日本]",[梅雨 [SEP] 梅雨（つゆ、ばいう）は、北海道と小笠原諸島を除く日本、朝鮮半島南部、中国...
4,jsquad-neural-search,1%3A0%3AAn_Z_ZgBGnBxNeQd9nv7,6.561416e-01,[梅雨がないとされている都道府県はどこ？],[北海道],[梅雨 [SEP] 実際の気象としては北海道にも道南を中心に梅雨前線がかかることはあるが、平...
5,jsquad-neural-search,1%3A0%3AqNzZ_ZgBQNa1Y6duygIE,5.894726e-01,[日本の地域で本格的な長雨に突入しない場所はどこか。],[北海道],[梅雨 [SEP] 次に梅雨前線は中国の江淮（長江流域・淮河流域）に北上する。6月下旬には華...
6,jsquad-neural-search,1%3A0%3As9zZ_ZgBQNa1Y6du2wJE,5.164980e-01,[年によっては梅雨入りの時期が特定できなかったり、あるいは発表がされないこともある。東・西日...,"[数年に一回, 数年に一回の割合, 数年に一回の割合で起こる]",[梅雨 [SEP] 年によっては梅雨入りの時期が特定できなかったり、あるいは発表がされないこ...
7,jsquad-neural-search,1%3A0%3Ak9zZ_ZgBQNa1Y6dupwIb,4.931808e-01,[梅雨は、世界的にどのあたりで見られる気象ですか？],"[東アジア, 東アジアの広範囲]",[梅雨 [SEP] 梅雨（つゆ、ばいう）は、北海道と小笠原諸島を除く日本、朝鮮半島南部、中国...
8,jsquad-neural-search,1%3A0%3A6X_Z_ZgBGnBxNeQdz3pc,4.886568e-01,[日本本土や朝鮮半島もどこから順に梅雨明けするか],"[南, 南から, 南から順に梅雨明け]",[梅雨 [SEP] 7月半ばを過ぎると、亜熱帯ジェット気流がチベット高原よりも北を流れるよう...
9,jsquad-neural-search,1%3A0%3A2Nza_ZgBQNa1Y6duFQLD,4.868331e-01,[梅雨の期間中ほとんど雨が降らない場合がある。このような梅雨のことをなんというか？],"[空梅雨, 空梅雨（からつゆ）]",[梅雨 [SEP] 梅雨の期間中ほとんど雨が降らない場合がある。このような梅雨のことを空梅雨...


ベクトル検索によって意図しない結果が付与されてしまう場合は、別のラボで解説するリランキングや、k ではなく min_score によるフィルタリングが有効です

<div class="alert alert-block alert-warning"> 
AOSS では、`response_processors.rerank` が使用できないため、リランキングは保留にしました。
</div>

In [47]:
%%time
index_name = "jsquad-neural-search"
query = "日本で梅雨がない地域は？"
payload = {
  "size": 10,
  "query": {
    "hybrid": {
      "queries": [
        {
          "match": {
            "question": {
              "query": query,
              "operator": "and"
            }
          }
        },
        {
          "neural": {
            "question_embedding": {
              "query_text": query, # テキストをベクトルに変換し
              "min_score": 0.7
            }
          }
        }
      ]
    }
  },
  "_source" : False,
  "fields": ["question", "answers",  "context"]
}
# 検索 API を実行
response = opensearch_client.search(
    body = payload,
    index = index_name,
    filter_path = "hits.hits",
    search_pipeline = hybrid_search_pipeline_id 
)

# 結果を表示
pd.json_normalize(response["hits"]["hits"])

CPU times: user 4.1 ms, sys: 0 ns, total: 4.1 ms
Wall time: 227 ms


,_index,_id,_score,fields.question,fields.answers,fields.context
0,jsquad-neural-search,1%3A0%3AktzZ_ZgBQNa1Y6duogLg,-9.549512e+09,[日本で梅雨がないのは北海道とどこか。],"[小笠原諸島, 小笠原諸島を除く日本]",[梅雨 [SEP] 梅雨（つゆ、ばいう）は、北海道と小笠原諸島を除く日本、朝鮮半島南部、中国...
1,jsquad-neural-search,1%3A0%3AktzZ_ZgBQNa1Y6duogLg,-4.422440e+09,[日本で梅雨がないのは北海道とどこか。],"[小笠原諸島, 小笠原諸島を除く日本]",[梅雨 [SEP] 梅雨（つゆ、ばいう）は、北海道と小笠原諸島を除く日本、朝鮮半島南部、中国...
2,jsquad-neural-search,1%3A0%3AktzZ_ZgBQNa1Y6duogLg,-4.422440e+09,[日本で梅雨がないのは北海道とどこか。],"[小笠原諸島, 小笠原諸島を除く日本]",[梅雨 [SEP] 梅雨（つゆ、ばいう）は、北海道と小笠原諸島を除く日本、朝鮮半島南部、中国...
3,jsquad-neural-search,1%3A0%3AktzZ_ZgBQNa1Y6duogLg,8.233960e-01,[日本で梅雨がないのは北海道とどこか。],"[小笠原諸島, 小笠原諸島を除く日本]",[梅雨 [SEP] 梅雨（つゆ、ばいう）は、北海道と小笠原諸島を除く日本、朝鮮半島南部、中国...
4,jsquad-neural-search,1%3A0%3AktzZ_ZgBQNa1Y6duogLg,-9.549512e+09,[日本で梅雨がないのは北海道とどこか。],"[小笠原諸島, 小笠原諸島を除く日本]",[梅雨 [SEP] 梅雨（つゆ、ばいう）は、北海道と小笠原諸島を除く日本、朝鮮半島南部、中国...
5,jsquad-neural-search,1%3A0%3AtNzZ_ZgBQNa1Y6du3ALV,-9.549512e+09,[梅雨が日本の中でない地域はどこか。],"[北海道, 東北地方]",[梅雨 [SEP] 年によっては梅雨明けの時期が特定できなかったり、あるいは発表がされないこ...
6,jsquad-neural-search,1%3A0%3AtNzZ_ZgBQNa1Y6du3ALV,-4.422440e+09,[梅雨が日本の中でない地域はどこか。],"[北海道, 東北地方]",[梅雨 [SEP] 年によっては梅雨明けの時期が特定できなかったり、あるいは発表がされないこ...
7,jsquad-neural-search,1%3A0%3AtNzZ_ZgBQNa1Y6du3ALV,-4.422440e+09,[梅雨が日本の中でない地域はどこか。],"[北海道, 東北地方]",[梅雨 [SEP] 年によっては梅雨明けの時期が特定できなかったり、あるいは発表がされないこ...
8,jsquad-neural-search,1%3A0%3AtNzZ_ZgBQNa1Y6du3ALV,8.682431e-01,[梅雨が日本の中でない地域はどこか。],"[北海道, 東北地方]",[梅雨 [SEP] 年によっては梅雨明けの時期が特定できなかったり、あるいは発表がされないこ...
9,jsquad-neural-search,1%3A0%3AtNzZ_ZgBQNa1Y6du3ALV,-9.549512e+09,[梅雨が日本の中でない地域はどこか。],"[北海道, 東北地方]",[梅雨 [SEP] 年によっては梅雨明けの時期が特定できなかったり、あるいは発表がされないこ...


### ベクトル検索をテキスト検索で補う
ベクトル検索は意味的に近い文書を検索することに長けていますが、反面厳密なマッチングができないケースがあります。例えば、製品の型番などの業務固有のパラメーターでの検索は不得手です。

以下のように "M" だけを検索対象としてみると、ベクトル検索は無関係の結果を返却します。

In [48]:
%%time
index_name = "jsquad-neural-search"
query = "M"
payload = {
  "size": 10,
  "query": {
    "neural": {
      "question_embedding": {
        "query_text": query, 
        "k": 10,
      }
    }
  },
  "_source" : False,
  "fields": ["question", "answers",  "context"]
}
# 検索 API を実行
response = opensearch_client.search(
    body = payload,
    index = index_name,
    filter_path = "hits.hits",
    search_pipeline = hybrid_search_pipeline_id 
)

# 結果を表示
pd.json_normalize(response["hits"]["hits"])

CPU times: user 4.04 ms, sys: 0 ns, total: 4.04 ms
Wall time: 233 ms


,_index,_id,_score,fields.question,fields.answers,fields.context
0,jsquad-neural-search,1%3A0%3AYNzk_ZgBQNa1Y6duGwnZ,0.430114,[ケプラーの式を導き出す際、Mが主星の質量、ｍが伴星の質量であるとするとき、ｍの影響は通常無...,"[Mはmよりもずっと大きいため, ここでGは重力定数、Mは主星の質量、mは伴星の質量である。...",[軌道長半径 [SEP] ここでGは重力定数、Mは主星の質量、mは伴星の質量である。通常、M...
1,jsquad-neural-search,1%3A0%3Ai9zi_ZgBQNa1Y6du0Qig,0.386891,[m5Cとは何か],[5-メチルシトシン],[核酸塩基 [SEP] DNAとRNAには、核酸の鎖が形成された後に修飾が行われた、非標準的...
2,jsquad-neural-search,1%3A0%3Ann_k_ZgBGnBxNeQdHIGq,0.383718,[軌道長半径において、Gは重力定数とするとMは],"[主星の質量, 伴星の質量]",[軌道長半径 [SEP] ここでGは重力定数、Mは主星の質量、mは伴星の質量である。通常、M...
3,jsquad-neural-search,1%3A0%3AWn_c_ZgBGnBxNeQdAHzM,0.376306,[ラオス語でムアンとは、郡という意味の他、どういう意味があるか],[街],[ラオス [SEP] ヴィエンチャン都と県の下には100前後の村（バーン）から成る郡（ムアン...
4,jsquad-neural-search,1%3A0%3AHdzg_ZgBQNa1Y6dupAdr,0.376103,[地球の中心から地表までの距離を R とすると、地球の中心から物体までの距離はどうなる],[R+h],[位置エネルギー [SEP] 今、地表から ｈ だけ高い質量 m の物体の位置エネルギーを考...
5,jsquad-neural-search,1%3A0%3AW3_c_ZgBGnBxNeQdAnxL,0.372645,[ムアンにはラオス語でどんな意味があるか],"[「郡」の他に「街」という意味, 郡]",[ラオス [SEP] ヴィエンチャン都と県の下には100前後の村（バーン）から成る郡（ムアン...
6,jsquad-neural-search,1%3A0%3APn_a_ZgBGnBxNeQdVHtF,0.371105,[「マーラーの奴、恥を知れ！（Shame on a man like Mahler!）」とい...,[アルトゥーロ・トスカニーニ],[グスタフ・マーラー [SEP] マーラーは演奏する曲に対して譜面に手を入れることが多く、ア...
7,jsquad-neural-search,1%3A0%3AU3_a_ZgBGnBxNeQddXvt,0.370046,[マーラーの交響曲作品はその規模の大きさや複雑さにも関わらず世界中のオーケストラにより頻繁に...,[ゲオルク・ショルティ],[グスタフ・マーラー [SEP] 現在、マーラーの交響曲作品はその規模の大きさや複雑さにも関...
8,jsquad-neural-search,1%3A0%3ABn_b_ZgBGnBxNeQdg3zR,0.369758,[国際連合事務総長だった人をあげよ],[ブトロス・ブトロス＝ガーリ],[国際連合平和維持活動 [SEP] また、活動の規模や内容も徐々に拡大・多様化し、1992年...
9,jsquad-neural-search,1%3A0%3AHdzc_ZgBQNa1Y6duAQSO,0.369108,[ムアンというラオス語の意味は何か],"[「郡」の他に「街」, 郡]",[ラオス [SEP] ヴィエンチャン都と県の下には100前後の村（バーン）から成る郡（ムアン...


一方、テキスト検索の方は M を含む結果を返します。

In [49]:
index_name = "jsquad-neural-search"
query = "M"

payload = {
  "query": {
    "match": {
      "question": {
        "query": query,
        "operator": "and"
      }
    }
  },
  "_source": False,
  "fields": ["question", "answers", "context"],
  "size": 10
}
response = opensearch_client.search(
    index=index_name,
    body=payload
)
pd.json_normalize(response["hits"]["hits"])

,_index,_id,_score,fields.question,fields.answers,fields.context
0,jsquad-neural-search,1%3A0%3Ai9zi_ZgBQNa1Y6du0Qig,8.735363,[m5Cとは何か],[5-メチルシトシン],[核酸塩基 [SEP] DNAとRNAには、核酸の鎖が形成された後に修飾が行われた、非標準的...
1,jsquad-neural-search,1%3A0%3A4dzh_ZgBQNa1Y6duzAcZ,7.457337,[I'm Feeling Lucky と書かれたボタンがあるホームページは],[Google],[Google_検索 [SEP] Googleのホームページには I'm Feeling L...
2,jsquad-neural-search,1%3A0%3AYNzk_ZgBQNa1Y6duGwnZ,7.127729,[ケプラーの式を導き出す際、Mが主星の質量、ｍが伴星の質量であるとするとき、ｍの影響は通常無...,"[Mはmよりもずっと大きいため, ここでGは重力定数、Mは主星の質量、mは伴星の質量である。...",[軌道長半径 [SEP] ここでGは重力定数、Mは主星の質量、mは伴星の質量である。通常、M...
3,jsquad-neural-search,1%3A0%3Ann_k_ZgBGnBxNeQdHIGq,6.233225,[軌道長半径において、Gは重力定数とするとMは],"[主星の質量, 伴星の質量]",[軌道長半径 [SEP] ここでGは重力定数、Mは主星の質量、mは伴星の質量である。通常、M...
4,jsquad-neural-search,1%3A0%3AHH_a_ZgBGnBxNeQdH3uo,5.608375,[地上付近では周囲から空気を吸い上げる一方、上空数千m-1万mの対流圏上層では吸い上げた空気...,"[台風や熱帯低気圧, 水蒸気, 豪雨]",[梅雨 [SEP] 台風や熱帯低気圧は地上付近では周囲から空気を吸い上げる一方、上空数千m-...
5,jsquad-neural-search,1%3A0%3A2X_l_ZgBGnBxNeQd84LB,5.201078,"[最高峰は島の北部にあるディアブロティン山で高さ1,447mである山がある国は？]",[ドミニカ国],[ドミニカ国 [SEP] ドミニカ島はカリブ海は小アンティル諸島にあり、北はグアドループ海峡...
6,jsquad-neural-search,1%3A0%3AWn_g_ZgBGnBxNeQdo3-v,3.773035,[地表から ｈ だけ高い質量 m の物体の位置エネルギーを考える。地球の中心から地表までの距...,[R+h],[位置エネルギー [SEP] 今、地表から ｈ だけ高い質量 m の物体の位置エネルギーを考...
7,jsquad-neural-search,1%3A0%3AHNzg_ZgBQNa1Y6duogfw,3.530683,[地表から ｈ だけ高い質量 m の物体の位置エネルギーを考えた場合、地球の中心から地表まで...,[R+h],[位置エネルギー [SEP] 今、地表から ｈ だけ高い質量 m の物体の位置エネルギーを考...


ベクトル検索単体からハイブリッド検索に切り替えることで、検索精度の向上を達成できます。

In [50]:
%%time
index_name = "jsquad-neural-search"
query = "M"
payload = {
  "size": 10,
  "query": {
    "hybrid": {
      "queries": [
        {
          "match": {
            "question": {
              "query": query,
              "operator": "and"
            }
          }
        },
        {
          "neural": {
            "question_embedding": {
              "query_text": query, # テキストをベクトルに変換し
              "min_score": 0.7
            }
          }
        }
      ]
    }
  },
  "_source" : False,
  "fields": ["question", "answers",  "context"]
}
# 検索 API を実行
response = opensearch_client.search(
    body = payload,
    index = index_name,
    filter_path = "hits.hits",
    search_pipeline = hybrid_search_pipeline_id 
)

# 結果を表示
pd.json_normalize(response["hits"]["hits"])

CPU times: user 4.41 ms, sys: 0 ns, total: 4.41 ms
Wall time: 260 ms


,_index,_id,_score,fields.question,fields.answers,fields.context
0,jsquad-neural-search,1%3A0%3Ai9zi_ZgBQNa1Y6du0Qig,-9.549512e+09,[m5Cとは何か],[5-メチルシトシン],[核酸塩基 [SEP] DNAとRNAには、核酸の鎖が形成された後に修飾が行われた、非標準的...
1,jsquad-neural-search,1%3A0%3Ai9zi_ZgBQNa1Y6du0Qig,-4.422440e+09,[m5Cとは何か],[5-メチルシトシン],[核酸塩基 [SEP] DNAとRNAには、核酸の鎖が形成された後に修飾が行われた、非標準的...
2,jsquad-neural-search,1%3A0%3Ai9zi_ZgBQNa1Y6du0Qig,8.735363e+00,[m5Cとは何か],[5-メチルシトシン],[核酸塩基 [SEP] DNAとRNAには、核酸の鎖が形成された後に修飾が行われた、非標準的...
3,jsquad-neural-search,1%3A0%3A4dzh_ZgBQNa1Y6duzAcZ,7.457337e+00,[I'm Feeling Lucky と書かれたボタンがあるホームページは],[Google],[Google_検索 [SEP] Googleのホームページには I'm Feeling L...
4,jsquad-neural-search,1%3A0%3Ai9zi_ZgBQNa1Y6du0Qig,-4.422440e+09,[m5Cとは何か],[5-メチルシトシン],[核酸塩基 [SEP] DNAとRNAには、核酸の鎖が形成された後に修飾が行われた、非標準的...
5,jsquad-neural-search,1%3A0%3Ai9zi_ZgBQNa1Y6du0Qig,-9.549512e+09,[m5Cとは何か],[5-メチルシトシン],[核酸塩基 [SEP] DNAとRNAには、核酸の鎖が形成された後に修飾が行われた、非標準的...
6,jsquad-neural-search,1%3A0%3AYNzk_ZgBQNa1Y6duGwnZ,-9.549512e+09,[ケプラーの式を導き出す際、Mが主星の質量、ｍが伴星の質量であるとするとき、ｍの影響は通常無...,"[Mはmよりもずっと大きいため, ここでGは重力定数、Mは主星の質量、mは伴星の質量である。...",[軌道長半径 [SEP] ここでGは重力定数、Mは主星の質量、mは伴星の質量である。通常、M...
7,jsquad-neural-search,1%3A0%3AYNzk_ZgBQNa1Y6duGwnZ,-4.422440e+09,[ケプラーの式を導き出す際、Mが主星の質量、ｍが伴星の質量であるとするとき、ｍの影響は通常無...,"[Mはmよりもずっと大きいため, ここでGは重力定数、Mは主星の質量、mは伴星の質量である。...",[軌道長半径 [SEP] ここでGは重力定数、Mは主星の質量、mは伴星の質量である。通常、M...
8,jsquad-neural-search,1%3A0%3AYNzk_ZgBQNa1Y6duGwnZ,7.127729e+00,[ケプラーの式を導き出す際、Mが主星の質量、ｍが伴星の質量であるとするとき、ｍの影響は通常無...,"[Mはmよりもずっと大きいため, ここでGは重力定数、Mは主星の質量、mは伴星の質量である。...",[軌道長半径 [SEP] ここでGは重力定数、Mは主星の質量、mは伴星の質量である。通常、M...
9,jsquad-neural-search,1%3A0%3Ann_k_ZgBGnBxNeQdHIGq,6.233225e+00,[軌道長半径において、Gは重力定数とするとMは],"[主星の質量, 伴星の質量]",[軌道長半径 [SEP] ここでGは重力定数、Mは主星の質量、mは伴星の質量である。通常、M...


### テキスト検索とスパース検索のハイブリッド検索
ベクトル検索の代わりに、スパース検索でテキスト検索を補完することも可能です。

### ベクトル検索とスパース検索のハイブリッド検索
ベクトル検索とスパース検索を組み合わせることも可能です。

<div class="alert alert-block alert-warning"> 
東京リージョンの Bedrock では、スパース検索が可能なモデルは使用できないので、スパース検索は除外しました。
</div>

## まとめ
ハイブリッド検索で様々な検索を組み合わせられることを確認してきました。ハイブリッド検索は必ずしもベクトル検索との組み合わせが必須ではなく、テキスト検索同士の組み合わせも可能です。様々な場面での利用を検討してみてください。

時間がある方は、続いて以下のラボも実施してみましょう。

- ~~[セマンティックリランキング (Amazon SageMaker 編)](../reranking/semantic-reranking-with-sagemaker.ipynb)~~

<div class="alert alert-block alert-warning"> 
AOSS では、`response_processors.rerank` が使用できないため、リランキングは保留にしました。
</div>

## 後片付け

### インデックス削除

In [51]:
index_name = "jsquad-hybrid-search"

try:
    response = opensearch_client.indices.delete(index=index_name)
    print(json.dumps(response, indent=2))
except Exception as e:
    print(e)

NotFoundError(404, 'index_not_found_exception', 'no such index [jsquad-hybrid-search]')
